In [307]:
import pandas as pd
import numpy as np
import re

In [388]:
reprisk_incidents = pd.read_csv('data/new_materiality_with_rri.csv') # Calculation in "CreateDataset" of "ESGmateriality" project
reprisk_incidents = reprisk_incidents[['gvkey', 'isin', 'cusip', 'reprisk_id', 'story_id', 'adopter', 'sic', 'SICS Codified Industry ', 'Codified SICS Sector ', 'severity', 'reach', 'novelty', 'incident_list', 'incident_date', 'cusip_cusip_tic', 'year', 'YearQuarter', 'materiality_reach', 'materiality_severity', 'materiality_car_1', 'materiality_car_5', 'materiality_rri', 'material_flag']]
reprisk_incidents

,gvkey,isin,cusip,reprisk_id,story_id,adopter,sic,SICS Codified Industry,Codified SICS Sector,severity,...,incident_date,cusip_cusip_tic,year,YearQuarter,materiality_reach,materiality_severity,materiality_car_1,materiality_car_5,materiality_rri,material_flag
0,213199.0,TW0002353000,NaN,10,826,1.0,3571.0,Hardware,Technology & Communications,2,...,2007-02-28,NaN,2007,2007Q1,0,0,0,0,0,1
1,213199.0,TW0002353000,NaN,10,1793,1.0,3571.0,Hardware,Technology & Communications,1,...,2007-09-09,NaN,2007,2007Q3,0,0,0,0,0,1
2,213199.0,TW0002353000,NaN,10,2335,1.0,3571.0,Hardware,Technology & Communications,1,...,2007-11-26,NaN,2007,2007Q4,1,0,0,0,0,0
3,213199.0,TW0002353000,NaN,10,2365,1.0,3571.0,Hardware,Technology & Communications,2,...,2007-08-21,NaN,2007,2007Q3,0,1,0,0,0,1
4,213199.0,TW0002353000,NaN,10,2513,1.0,3571.0,Hardware,Technology & Communications,1,...,2007-11-30,NaN,2007,2007Q4,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295249,295326.0,CNE100000TP3,NaN,99984,8294414,1.0,3678.0,Electrical & Electronic Equipment,Resource Transformation,1,...,2022-08-30,NaN,2022,2022Q3,0,0,0,0,1,1
295250,295326.0,CNE100000TP3,NaN,99984,8339482,1.0,3678.0,Electrical & Electronic Equipment,Resource Transformation,1,...,2023-01-01,NaN,2023,2023Q1,0,0,1,1,0,0
295251,295326.0,CNE100000TP3,NaN,99984,8362588,1.0,3678.0,Electrical & Electronic Equipment,Resource Transformation,1,...,2022-10-28,NaN,2022,2022Q4,0,0,0,0,0,0
295252,295326.0,CNE100000TP3,NaN,99984,8376615,1.0,3678.0,Electrical & Electronic Equipment,Resource Transformation,2,...,2023-04-20,NaN,2023,2023Q2,1,0,0,0,0,1


In [21]:
df = reprisk_incidents.copy()

# Ensure proper quarterly Periods
df['YearQuarter'] = pd.PeriodIndex(df['YearQuarter'].astype(str), freq='Q')

# Coerce flags to numeric and treat missing as 0
flag_cols = [
    'material_flag',
    'materiality_car_1', 'materiality_car_5',
    'materiality_rri', 'materiality_reach', 'materiality_severity'
]
df[flag_cols] = df[flag_cols].apply(pd.to_numeric, errors='coerce').fillna(0)

key = ["gvkey", "YearQuarter"]

# 1) Aggregate incidents per firm-quarter
agg_fq = (
    df.groupby(key, dropna=False)
      .agg(
          n_material          = ('material_flag',         lambda s: (s == 1).sum()),
          n_nonmaterial       = ('material_flag',         lambda s: (s == 0).sum()),
          n_car_1_material    = ('materiality_car_1',     lambda s: (s == 1).sum()),
          n_car_5_material    = ('materiality_car_5',     lambda s: (s == 1).sum()),
          n_rri_material      = ('materiality_rri',       lambda s: (s == 1).sum()),
          n_reach_material    = ('materiality_reach',     lambda s: (s == 1).sum()),
          n_severity_material = ('materiality_severity',  lambda s: (s == 1).sum()),
      )
      .reset_index()
)

# 2) Build the BALANCED universe = all firms × all quarters (global min..max)
firms = (
    df['gvkey']
    .dropna()
    .unique()
)
qmin = df['YearQuarter'].min()
qmax = df['YearQuarter'].max()
quarters = pd.period_range(qmin, qmax, freq='Q')

balanced_idx = pd.MultiIndex.from_product([firms, quarters], names=key)
balanced_universe = pd.DataFrame(index=balanced_idx).reset_index()

# 3) Merge counts onto balanced universe and fill missing with zeros
reprisk = (
    balanced_universe
      .merge(agg_fq, on=key, how='left')
      .fillna(0)
      .sort_values(key)
      .reset_index(drop=True)
)

# 4) Cast counts to ints
count_cols = [
    'n_material','n_nonmaterial','n_car_1_material','n_car_5_material',
    'n_rri_material','n_reach_material','n_severity_material'
]
reprisk[count_cols] = reprisk[count_cols].astype('int16')

# (optional) If you prefer strings for YearQuarter:
reprisk['YearQuarter'] = reprisk['YearQuarter'].astype(str)

reprisk

,gvkey,YearQuarter,n_material,n_nonmaterial,n_car_1_material,n_car_5_material,n_rri_material,n_reach_material,n_severity_material
0,1004.0,2007Q1,0,0,0,0,0,0,0
1,1004.0,2007Q2,0,0,0,0,0,0,0
2,1004.0,2007Q3,0,0,0,0,0,0,0
3,1004.0,2007Q4,0,0,0,0,0,0,0
4,1004.0,2008Q1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
1091463,367430.0,2022Q4,0,0,0,0,0,0,0
1091464,367430.0,2023Q1,0,0,0,0,0,0,0
1091465,367430.0,2023Q2,0,0,0,0,0,0,0
1091466,367430.0,2023Q3,0,0,0,0,0,0,0


In [24]:
reprisk_merged = reprisk.merge(reprisk_incidents[['gvkey', 'cusip', 'SICS Codified Industry ', 'Codified SICS Sector ']].drop_duplicates(), on='gvkey', how='left')
reprisk_merged

,gvkey,YearQuarter,n_material,n_nonmaterial,n_car_1_material,n_car_5_material,n_rri_material,n_reach_material,n_severity_material,cusip,SICS Codified Industry,Codified SICS Sector
0,1004.0,2007Q1,0,0,0,0,0,0,0,00036110,Industrial Machinery & Goods,Resource Transformation
1,1004.0,2007Q2,0,0,0,0,0,0,0,00036110,Industrial Machinery & Goods,Resource Transformation
2,1004.0,2007Q3,0,0,0,0,0,0,0,00036110,Industrial Machinery & Goods,Resource Transformation
3,1004.0,2007Q4,0,0,0,0,0,0,0,00036110,Industrial Machinery & Goods,Resource Transformation
4,1004.0,2008Q1,0,0,0,0,0,0,0,00036110,Industrial Machinery & Goods,Resource Transformation
...,...,...,...,...,...,...,...,...,...,...,...,...
1091463,367430.0,2022Q4,0,0,0,0,0,0,0,NaN,Agricultural Products,Food & Beverage
1091464,367430.0,2023Q1,0,0,0,0,0,0,0,NaN,Agricultural Products,Food & Beverage
1091465,367430.0,2023Q2,0,0,0,0,0,0,0,NaN,Agricultural Products,Food & Beverage
1091466,367430.0,2023Q3,0,0,0,0,0,0,0,NaN,Agricultural Products,Food & Beverage


In [28]:
df = reprisk_incidents.copy()

# --- Use existing year column ---
# Make sure it's integer-like
df['year'] = pd.to_numeric(df['year'], errors='coerce').astype('Int64')

key_y = ['gvkey', 'year']

# --- Incident counts per firm–year ---
counts = (
    df.groupby(key_y, dropna=False)
      .agg(
          n_material          = ('material_flag',        lambda s: (s == 1).sum()),
          n_nonmaterial       = ('material_flag',        lambda s: (s == 0).sum()),
          n_car_1_material    = ('materiality_car_1',    lambda s: (s == 1).sum()),
          n_car_5_material    = ('materiality_car_5',    lambda s: (s == 1).sum()),
          n_rri_material      = ('materiality_rri',      lambda s: (s == 1).sum()),
          n_reach_material    = ('materiality_reach',    lambda s: (s == 1).sum()),
          n_severity_material = ('materiality_severity', lambda s: (s == 1).sum()),
      )
      .reset_index()
)

# --- Balanced universe: all firms × all years (global min..max of df['year']) ---
firms = df['gvkey'].dropna().unique()
ymin  = int(df['year'].min())
ymax  = int(df['year'].max())
years = pd.Index(range(ymin, ymax + 1), name='year')

balanced = (
    pd.MultiIndex.from_product([firms, years], names=key_y)
      .to_frame(index=False)
)

# --- Merge counts; zeros where no incidents ---
reprisk_yearly = (
    balanced.merge(counts, on=key_y, how='left')
            .fillna(0)
            .sort_values(key_y)
            .reset_index(drop=True)
)

count_cols = [
    'n_material','n_nonmaterial','n_car_1_material','n_car_5_material',
    'n_rri_material','n_reach_material','n_severity_material'
]
reprisk_yearly[count_cols] = reprisk_yearly[count_cols].astype('int16')


reprisk_yearly


,gvkey,year,n_material,n_nonmaterial,n_car_1_material,n_car_5_material,n_rri_material,n_reach_material,n_severity_material
0,1004.0,2007,0,0,0,0,0,0,0
1,1004.0,2008,0,0,0,0,0,0,0
2,1004.0,2009,0,0,0,0,0,0,0
3,1004.0,2010,0,1,0,0,0,1,0
4,1004.0,2011,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
272862,367430.0,2019,0,0,0,0,0,0,0
272863,367430.0,2020,0,0,0,0,0,0,0
272864,367430.0,2021,0,0,0,0,0,0,0
272865,367430.0,2022,0,0,0,0,0,0,0


In [29]:
reprisk_merged_yearly = reprisk_yearly.merge(reprisk_incidents[['gvkey', 'cusip', 'SICS Codified Industry ', 'Codified SICS Sector ']].drop_duplicates(), on='gvkey', how='left')
reprisk_merged_yearly

,gvkey,year,n_material,n_nonmaterial,n_car_1_material,n_car_5_material,n_rri_material,n_reach_material,n_severity_material,cusip,SICS Codified Industry,Codified SICS Sector
0,1004.0,2007,0,0,0,0,0,0,0,00036110,Industrial Machinery & Goods,Resource Transformation
1,1004.0,2008,0,0,0,0,0,0,0,00036110,Industrial Machinery & Goods,Resource Transformation
2,1004.0,2009,0,0,0,0,0,0,0,00036110,Industrial Machinery & Goods,Resource Transformation
3,1004.0,2010,0,1,0,0,0,1,0,00036110,Industrial Machinery & Goods,Resource Transformation
4,1004.0,2011,0,0,0,0,0,0,0,00036110,Industrial Machinery & Goods,Resource Transformation
...,...,...,...,...,...,...,...,...,...,...,...,...
272862,367430.0,2019,0,0,0,0,0,0,0,NaN,Agricultural Products,Food & Beverage
272863,367430.0,2020,0,0,0,0,0,0,0,NaN,Agricultural Products,Food & Beverage
272864,367430.0,2021,0,0,0,0,0,0,0,NaN,Agricultural Products,Food & Beverage
272865,367430.0,2022,0,0,0,0,0,0,0,NaN,Agricultural Products,Food & Beverage


In [394]:
reprisk_merged_yearly.groupby('year')['cusip'].nunique()

year
2007    4243
2008    4243
2009    4243
2010    4243
2011    4243
2012    4243
2013    4243
2014    4243
2015    4243
2016    4243
2017    4243
2018    4243
2019    4243
2020    4243
2021    4243
2022    4243
2023    4243
Name: cusip, dtype: int64

In [395]:
reprisk_merged_yearly.notna().sum()

gvkey                      272867
year                       272867
n_material                 272867
n_nonmaterial              272867
n_car_1_material           272867
n_car_5_material           272867
n_rri_material             272867
n_reach_material           272867
n_severity_material        272867
cusip                       72131
SICS Codified Industry     272867
Codified SICS Sector       272867
dtype: int64

# 1. Portfolios based on material incidents (yes/no)

In [288]:
# df = your yearly panel
df = reprisk_merged_yearly.copy()
df.drop_duplicates(['cusip', 'year'], keep='last', inplace=True)
df['year'] = pd.to_numeric(df['year'], errors='coerce').astype('Int64')

metrics = {
    'material': 'n_material',
    'car_1':    'n_car_1_material',
    'car_5':    'n_car_5_material',
    'reach':    'n_reach_material',
    'severity': 'n_severity_material',
}

# --- Long memberships (one row per cusip-year-metric) ---
frames = []
for mname, mcol in metrics.items():
    tmp = df[['cusip', 'year', mcol]].copy()
    tmp['metric'] = mname
    tmp['portfolio'] = np.where(tmp[mcol] > 0, 'incident', 'no_incident')
    frames.append(tmp[['cusip', 'year', 'metric', 'portfolio']])

portfolios_long = pd.concat(frames, ignore_index=True)

# --- Wide memberships (one row per cusip-year; one column per metric) ---
wide = df[['cusip', 'year']].drop_duplicates().sort_values(['cusip', 'year']).copy()
wide_idx = wide.set_index(['cusip', 'year']).index

for mname, mcol in metrics.items():
    # make a Series aligned on (gvkey, year) index
    s = df.set_index(['cusip', 'year'])[mcol]
    s = (s > 0).map({True: 'incident', False: 'no_incident'})
    # align to the wide index
    wide[f'{mname}_portfolio'] = s.reindex(wide_idx).to_numpy()

# Example: counts per year & portfolio
portfolio_counts = (
    portfolios_long
    .groupby(['metric', 'year', 'portfolio'], as_index=False)
    .agg(n_firms=('cusip', 'nunique'))
    .sort_values(['metric', 'year', 'portfolio'])
)


In [289]:
def build_incident_flags(df):
    out = df.copy()

    # 1) find all portfolio label columns
    port_cols = [c for c in out.columns if c.endswith('_portfolio')]

    # 3) create 0/1 flags per portfolio column
    #    naming: noInc_<base>, inc_<base>  (e.g., noInc_material, inc_material)
    for c in port_cols:
        base = c.replace('_portfolio', '')  # e.g., 'material'
        out[f'noInc_{base}'] = (out[c] == 'no_incident').astype('int8')
        out[f'inc_{base}']   = (out[c] == 'incident').astype('int8')


    return out

In [290]:
# ---- use it ----
wide_flags = build_incident_flags(wide)  # replace df_inc with your DataFrame name

wide_flags


,cusip,year,material_portfolio,car_1_portfolio,car_5_portfolio,reach_portfolio,severity_portfolio,noInc_material,inc_material,noInc_car_1,inc_car_1,noInc_car_5,inc_car_5,noInc_reach,inc_reach,noInc_severity,inc_severity
81991,00030710,2007,no_incident,no_incident,no_incident,no_incident,no_incident,1,0,1,0,1,0,1,0,1,0
81992,00030710,2008,no_incident,no_incident,no_incident,no_incident,no_incident,1,0,1,0,1,0,1,0,1,0
81993,00030710,2009,no_incident,no_incident,no_incident,no_incident,no_incident,1,0,1,0,1,0,1,0,1,0
81994,00030710,2010,no_incident,no_incident,no_incident,no_incident,no_incident,1,0,1,0,1,0,1,0,1,0
81995,00030710,2011,no_incident,no_incident,no_incident,no_incident,no_incident,1,0,1,0,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272862,NaN,2019,no_incident,no_incident,no_incident,no_incident,no_incident,1,0,1,0,1,0,1,0,1,0
272863,NaN,2020,no_incident,no_incident,no_incident,no_incident,no_incident,1,0,1,0,1,0,1,0,1,0
272864,NaN,2021,no_incident,no_incident,no_incident,no_incident,no_incident,1,0,1,0,1,0,1,0,1,0
272865,NaN,2022,no_incident,no_incident,no_incident,no_incident,no_incident,1,0,1,0,1,0,1,0,1,0


# 2. Get financials

In [130]:
reprisk_merged_yearly = pd.read_csv('Data/incidents_rolling_industry_all.csv')
reprisk_merged_yearly

,gvkey,YearMonth,n_material,n_nonmaterial,n_car_1_material,n_car_5_material,n_reach_material,n_severity_material,cusip,SICS Codified Industry,...,industry_n_car_1_material,industry_n_car_5_material,industry_n_reach_material,industry_n_severity_material,date,mktrf,smb,hml,rf,umd
0,1004.0,2007-01-01,0,0,0,0,0,0,000361105,Industrial Machinery & Goods,...,0,0,5,5,2007-01-01,0.000138,0.000009,-0.000068,0.000044,0.000017
1,1004.0,2007-02-01,0,0,0,0,0,0,000361105,Industrial Machinery & Goods,...,2,2,7,7,2007-02-01,-0.000196,0.000123,-0.000008,0.000038,-0.000129
2,1004.0,2007-03-01,0,0,0,0,0,0,000361105,Industrial Machinery & Goods,...,1,1,6,6,2007-03-01,0.000071,0.000009,-0.000088,0.000043,0.000258
3,1004.0,2007-04-01,0,0,0,0,0,0,000361105,Industrial Machinery & Goods,...,0,0,7,7,2007-04-01,0.000349,-0.000221,-0.000144,0.000044,-0.000016
4,1004.0,2007-05-01,0,0,0,0,0,0,000361105,Industrial Machinery & Goods,...,1,1,8,9,2007-05-01,0.000323,0.000016,-0.000059,0.000041,-0.000033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2390467,356128.0,2023-08-01,0,0,0,0,0,0,48581R205,Consumer Finance,...,3,3,14,14,2023-08-01,-0.000236,-0.000314,-0.000115,0.000045,0.000378
2390468,356128.0,2023-09-01,0,0,0,0,0,0,48581R205,Consumer Finance,...,0,0,10,10,2023-09-01,-0.000523,-0.000251,0.000149,0.000043,0.000033
2390469,356128.0,2023-10-01,0,0,0,0,0,0,48581R205,Consumer Finance,...,0,0,7,7,2023-10-01,-0.000315,-0.000386,0.000020,0.000047,0.000167
2390470,356128.0,2023-11-01,0,0,0,0,0,0,48581R205,Consumer Finance,...,1,0,1,1,2023-11-01,0.000888,-0.000010,0.000167,0.000044,0.000256


In [131]:
reprisk_merged_yearly['YearQuarter'] = pd.to_datetime(reprisk_merged_yearly['YearMonth']).dt.to_period('Q')
reprisk_merged_yearly['YearQuarter_prior'] = reprisk_merged_yearly['YearQuarter'] - 1
reprisk_merged_yearly['YearQuarter_prior'] = reprisk_merged_yearly['YearQuarter_prior'].astype(str)

In [26]:
# Export unique gvkey as int to txt
reprisk_merged_yearly['gvkey'].drop_duplicates().astype(str).str.replace(r'\.0$', '', regex=True).to_csv('data/gvkeys.txt', index=False, header=False)

In [207]:
import wrds
from pathlib import Path
from dotenv import load_dotenv
import os

project_root = Path(r"E:\GermanBusinessPanelTeam\Schrader\Forschung\ESGmateriality")

# ── Load WRDS creds from wrds.env in the project root ─────────────────────────
env_path = project_root / "wrds.env"
load_dotenv(dotenv_path=env_path)
# ── Connect to WRDS ───────────────────────────────────────────────────────────
db = wrds.Connection(
    wrds_username=os.getenv("WRDS_YALE_USERNAME"),
    wrds_password=os.getenv("WRDS_YALE_PASSWORD")
)

Loading library list...
Done


In [41]:
# ---------- tiny helper: keys + single Compustat QUARTERLY pull ----------

def _parse_yearquarter(s):
    """
    Accepts strings like '2022Q3', '2022-Q3', '2022 Q3', Periods, or timestamps.
    Returns a pandas.Period (freq='Q-DEC').
    """
    if pd.isna(s):
        return pd.NaT
    if isinstance(s, pd.Period):
        return s.asfreq('Q-DEC')
    s = str(s).strip().upper().replace('-', '').replace(' ', '')
    if 'Q' in s:
        y, q = s.split('Q', 1)
        return pd.Period(year=int(y), quarter=int(q), freq='Q-DEC')
    try:
        dt = pd.to_datetime(s, errors='raise')
        return dt.to_period('Q-DEC')
    except Exception:
        return pd.NaT

def _keys(df):
    """
    For quarterly pulls:
      - expects columns ['gvkey', 'YearQuarter_prior'] (e.g., '2024Q3')
      - returns distinct gvkey + quarter keys, also provides 'year' and 'quarter'
    """
    out = (df[['gvkey', 'YearQuarter_prior']].dropna()
           .assign(
               gvkey=lambda d: d['gvkey'].astype(str).str.replace(r'\.0$', '', regex=True).str.zfill(6),
               qtr  =lambda d: d['YearQuarter_prior'].map(_parse_yearquarter))
           .dropna(subset=['qtr'])
           .drop_duplicates())
    out['year']    = out['qtr'].dt.year.astype(int)
    out['quarter'] = out['qtr'].dt.quarter.astype(int)
    return out[['gvkey','qtr','year','quarter']]

def _pull_compustat_quarterly(db, keys):
    """
    Pull ONLY the variables needed for:
      - Market cap / Size: prccq, cshoq
      - ROA: ibq, atq
      - Leverage: ltq, atq
      - Book-to-Market: seqq, ceqq, pstkq, txditcq

    Returns compq with calendar quarter fields and a normalized join key (gvkey_norm).
    """
    import pandas as pd

    if keys.empty:
        return pd.DataFrame(columns=[
            'gvkey','datadate','prccq','cshoq','ibq','atq','ltq','seqq','ceqq','pstkq','txditcq',
            'calyear','calqtr','calyrqtr','gvkey_norm'
        ])

    # window: from (min quarter - 1) to max quarter (safe & small)
    min_q = (keys['qtr'].min() - 1)
    max_q = keys['qtr'].max()
    start_date = min_q.asfreq('Q-DEC').to_timestamp(how='end').normalize()
    end_date   = max_q.asfreq('Q-DEC').to_timestamp(how='end').normalize()

    gvkeys = tuple(keys['gvkey'].unique().tolist())  # already normalized in _keys()

    compq = db.raw_sql("""
        SELECT gvkey, datadate,
               prccq, cshoq,           -- market cap
               ibq, atq,               -- ROA
               ltq,                    -- leverage
               seqq, ceqq, pstkq, txditcq  -- book equity pieces
        FROM comp.fundq
        WHERE indfmt='INDL' AND datafmt='STD' AND consol='C'
          AND gvkey IN %(g)s
          AND datadate BETWEEN %(a)s AND %(b)s
    """, params={
        "g": gvkeys,
        "a": pd.Timestamp(start_date),
        "b": pd.Timestamp(end_date)
    }, date_cols=['datadate'])

    if compq.empty:
        return pd.DataFrame(columns=[
            'gvkey','datadate','prccq','cshoq','ibq','atq','ltq','seqq','ceqq','pstkq','txditcq',
            'calyear','calqtr','calyrqtr','gvkey_norm'
        ])

    # calendar fields + normalized gvkey (for joining)
    compq['datadate'] = pd.to_datetime(compq['datadate'])
    compq['calyear']  = compq['datadate'].dt.year.astype('Int64')
    compq['calqtr']   = compq['datadate'].dt.quarter.astype('Int64')
    compq['calyrqtr'] = compq['calyear'].astype(str) + 'Q' + compq['calqtr'].astype(str)
    compq['gvkey_norm'] = compq['gvkey'].astype(str).str.replace(r'\.0$', '', regex=True).str.zfill(6)
    return compq


In [51]:
# --- collapse comp.fundq to one row per (gvkey, quarter) and compute ratios ---
def _compute_quarterly_metrics(compq: pd.DataFrame) -> pd.DataFrame:

    if compq.empty:
        return pd.DataFrame(columns=['gvkey_norm','calyrqtr','size','roa_q','lev_q','bm_q'])

    compq = (compq.sort_values(['gvkey_norm','calyrqtr','datadate'])
                  .drop_duplicates(['gvkey_norm','calyrqtr'], keep='last')).copy()

    # to numeric Series
    prccq   = pd.to_numeric(compq['prccq'],   errors='coerce')
    cshoq   = pd.to_numeric(compq['cshoq'],   errors='coerce')
    ibq     = pd.to_numeric(compq['ibq'],     errors='coerce')
    atq     = pd.to_numeric(compq['atq'],     errors='coerce')
    ltq     = pd.to_numeric(compq['ltq'],     errors='coerce')
    seqq    = pd.to_numeric(compq['seqq'],    errors='coerce')
    ceqq    = pd.to_numeric(compq['ceqq'],    errors='coerce')
    pstkq   = pd.to_numeric(compq['pstkq'],   errors='coerce')
    txditcq = pd.to_numeric(compq['txditcq'], errors='coerce')

    # numpy views (avoid masked ufunc path)
    prc   = prccq.to_numpy(dtype='float64')
    csho  = cshoq.to_numpy(dtype='float64')
    ib    = ibq.to_numpy(dtype='float64')
    at    = atq.to_numpy(dtype='float64')
    lt    = ltq.to_numpy(dtype='float64')
    seq   = seqq.to_numpy(dtype='float64')
    ceq   = ceqq.to_numpy(dtype='float64')
    pstk  = pstkq.to_numpy(dtype='float64')
    txd   = txditcq.to_numpy(dtype='float64')

    # market cap
    mktcap = prc * csho

    # size = ln(mktcap) only if mktcap > 0
    size = np.where(mktcap > 0, np.log(mktcap), np.nan)

    # ROA and Leverage: safe divisions
    roa_q = np.divide(ib, at, out=np.full_like(at, np.nan), where=(at != 0) & ~np.isnan(at))
    lev_q = np.divide(lt, at, out=np.full_like(at, np.nan), where=(at != 0) & ~np.isnan(at))

    # Book equity: BE = (SEQ if present else CEQ) + TXDITCQ − PSTKQ
    seq_filled = np.where(~np.isnan(seq), seq, ceq)
    be = seq_filled + txd - pstk

    # Book-to-Market: only if mktcap > 0
    bm_q = np.divide(be, mktcap, out=np.full_like(mktcap, np.nan), where=(mktcap > 0) & ~np.isnan(mktcap))

    return pd.DataFrame({
        'gvkey_norm': compq['gvkey_norm'].values,
        'calyrqtr':   compq['calyrqtr'].values,
        'size':       size,
        'roa_q':      roa_q,
        'lev_q':      lev_q,
        'bm_q':       bm_q
    })


# --- build mapping (gvkey_norm, YearQuarter_prior) -> metrics ---
def _get_quarterly_metrics_map(db, df_quarterly: pd.DataFrame) -> pd.DataFrame:
    # keys with normalized gvkey + Period quarter (used for pull)
    k0 = _keys(df_quarterly)
    if k0.empty:
        return pd.DataFrame(columns=['gvkey_norm','YearQuarter_prior','size','roa_q','lev_q','bm_q'])

    # pull only once over the window
    compq = _pull_compustat_quarterly(db, k0)

    # compute metrics at (gvkey_norm, calyrqtr)
    metrics = _compute_quarterly_metrics(compq)

    # make left-side merge keys from k0 (string quarter like '2007Q1')
    left_keys = (k0.assign(
        YearQuarter_prior=k0['qtr'].astype(str),
        calyrqtr=k0['qtr'].dt.year.astype(str) + 'Q' + k0['qtr'].dt.quarter.astype(str),
        gvkey_norm=k0['gvkey'])
        [['gvkey_norm','YearQuarter_prior','calyrqtr']].drop_duplicates()
    )

    # map metrics to (gvkey_norm, YearQuarter_prior)
    out = left_keys.merge(metrics, on=['gvkey_norm','calyrqtr'], how='left')
    return out[['gvkey_norm','YearQuarter_prior','size','roa_q','lev_q','bm_q']]

# --- public: add quarterly metrics back to your original dataframe ---
def add_quarterly_metrics(db, reprisk_merged_quarterly: pd.DataFrame) -> pd.DataFrame:
    df = reprisk_merged_quarterly.copy()

    # temp normalized key for joining (same as _keys)
    df['gvkey_norm'] = df['gvkey'].astype(str).str.replace(r'\.0$', '', regex=True).str.zfill(6)

    metrics_map = _get_quarterly_metrics_map(db, df)

    out = df.merge(metrics_map,
                   on=['gvkey_norm','YearQuarter_prior'],
                   how='left')

    return out.drop(columns=['gvkey_norm'])

In [52]:
reprisk_with_metrics = add_quarterly_metrics(db, reprisk_merged_yearly)
reprisk_with_metrics

C:\Users\PascalSchrader.WIN-T1MVMVKE25U.000\AppData\Local\Temp\9\ipykernel_10332\93899279.py:36: RuntimeWarning: divide by zero encountered in log
  size = np.where(mktcap > 0, np.log(mktcap), np.nan)


,gvkey,YearMonth,n_material,n_nonmaterial,n_car_1_material,n_car_5_material,n_reach_material,n_severity_material,cusip,SICS Codified Industry,...,smb,hml,rf,umd,YearQuarter,YearQuarter_prior,size,roa_q,lev_q,bm_q
0,1004.0,2007-01-01,0,0,0,0,0,0,000361105,Industrial Machinery & Goods,...,0.000009,-0.000068,0.000044,0.000017,2007Q1,2006Q4,6.889099,0.013968,0.549214,0.495846
1,1004.0,2007-02-01,0,0,0,0,0,0,000361105,Industrial Machinery & Goods,...,0.000123,-0.000008,0.000038,-0.000129,2007Q1,2006Q4,6.889099,0.013968,0.549214,0.495846
2,1004.0,2007-03-01,0,0,0,0,0,0,000361105,Industrial Machinery & Goods,...,0.000009,-0.000088,0.000043,0.000258,2007Q1,2006Q4,6.889099,0.013968,0.549214,0.495846
3,1004.0,2007-04-01,0,0,0,0,0,0,000361105,Industrial Machinery & Goods,...,-0.000221,-0.000144,0.000044,-0.000016,2007Q2,2007Q1,6.987112,0.015352,0.535340,0.468633
4,1004.0,2007-05-01,0,0,0,0,0,0,000361105,Industrial Machinery & Goods,...,0.000016,-0.000059,0.000041,-0.000033,2007Q2,2007Q1,6.987112,0.015352,0.535340,0.468633
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2390467,356128.0,2023-08-01,0,0,0,0,0,0,48581R205,Consumer Finance,...,-0.000314,-0.000115,0.000045,0.000378,2023Q3,2023Q2,NaN,NaN,NaN,NaN
2390468,356128.0,2023-09-01,0,0,0,0,0,0,48581R205,Consumer Finance,...,-0.000251,0.000149,0.000043,0.000033,2023Q3,2023Q2,NaN,NaN,NaN,NaN
2390469,356128.0,2023-10-01,0,0,0,0,0,0,48581R205,Consumer Finance,...,-0.000386,0.000020,0.000047,0.000167,2023Q4,2023Q3,9.846527,0.034406,0.833953,NaN
2390470,356128.0,2023-11-01,0,0,0,0,0,0,48581R205,Consumer Finance,...,-0.000010,0.000167,0.000044,0.000256,2023Q4,2023Q3,9.846527,0.034406,0.833953,NaN


In [53]:
reprisk_with_metrics.isna().sum()

gvkey                                 0
YearMonth                             0
n_material                            0
n_nonmaterial                         0
n_car_1_material                      0
n_car_5_material                      0
n_reach_material                      0
n_severity_material                   0
cusip                                 0
SICS Codified Industry                0
Codified SICS Sector                  0
year                                  0
industry_n_material                   0
industry_n_car_1_material             0
industry_n_car_5_material             0
industry_n_reach_material             0
industry_n_severity_material          0
date                                  0
mktrf                                 0
smb                                   0
hml                                   0
rf                                    0
umd                                   0
YearQuarter                           0
YearQuarter_prior                     0


In [821]:
# ---------- 4) R&D intensity ----------

def get_rd_intensity(db, reprisk_merged_yearly):
    """
    R&D intensity = xrd / sale using year-1 values.
    """
    keys = _keys(reprisk_merged_yearly)
    if keys.empty:
        return pd.DataFrame(columns=['gvkey','year','rd_intensity'])
    comp = _pull_compustat(db, keys)

    fy1 = (comp.assign(year=lambda d: d['calyear'] + 1)
                .sort_values(['gvkey','year','datadate'])
                .drop_duplicates(['gvkey','year'], keep='last')
                [['gvkey','year','xrd','sale']])

    sale_pos = fy1['sale'].where(fy1['sale'] > 0, np.nan)
    out = fy1[['gvkey','year']].copy()
    out['rd_intensity'] = fy1['xrd'] / sale_pos
    return keys.merge(out, on=['gvkey','year'], how='left')

In [822]:
rd_df = get_rd_intensity(db, keys_df)
rd_df.notna().sum()

gvkey           273598
year            273598
rd_intensity     22968
dtype: int64

In [823]:
# ---------- 5) Advertising intensity ----------

def get_ad_intensity(db, reprisk_merged_yearly):
    """
    Advertising intensity = xad / sale using year-1 values.
    """
    keys = _keys(reprisk_merged_yearly)
    if keys.empty:
        return pd.DataFrame(columns=['gvkey','year','ad_intensity'])
    comp = _pull_compustat(db, keys)

    fy1 = (comp.assign(year=lambda d: d['calyear'] + 1)
                .sort_values(['gvkey','year','datadate'])
                .drop_duplicates(['gvkey','year'], keep='last')
                [['gvkey','year','xad','sale']])

    sale_pos = fy1['sale'].where(fy1['sale'] > 0, np.nan)
    out = fy1[['gvkey','year']].copy()
    out['ad_intensity'] = fy1['xad'] / sale_pos
    return keys.merge(out, on=['gvkey','year'], how='left')

In [824]:
ad_df = get_ad_intensity(db, keys_df)
ad_df.notna().sum()

gvkey           273598
year            273598
ad_intensity     19305
dtype: int64

In [152]:
import pandas as pd

def pull_compustat_quarterly_min(db, start="1990-01-01", end="2025-12-31"):
    sql = f"""
        SELECT
            gvkey,
            datadate::date AS datadate
        FROM comp.fundq
        WHERE datadate BETWEEN DATE '{start}' AND DATE '{end}'
          AND indfmt = 'INDL'
          AND datafmt = 'STD'
          AND popsrc = 'D'
          AND consol = 'C'
    """
    return db.raw_sql(sql, date_cols=['datadate']).sort_values(['gvkey','datadate']).reset_index(drop=True)

# Example pull
df_min = pull_compustat_quarterly_min(db, start="2007-01-01", end="2025-12-31")

# --- coverage stats ---
# total distinct firms
n_firms_total = df_min['gvkey'].nunique()

# by calendar year
firms_by_year = (
    df_min.assign(year=df_min['datadate'].dt.year)
          .groupby('year')['gvkey'].nunique()
          .rename('n_firms')
          .reset_index()
)

# by calendar quarter
firms_by_quarter = (
    df_min.assign(yq=df_min['datadate'].dt.to_period('Q'))
          .groupby('yq')['gvkey'].nunique()
          .rename('n_firms')
          .reset_index()
)

print("Distinct firms (total):", n_firms_total)
print(firms_by_year.head())
print(firms_by_quarter.head())


Distinct firms (total): 27108
   year  n_firms
0  2007    11529
1  2008    11376
2  2009    11255
3  2010    11368
4  2011    11680
       yq  n_firms
0  2007Q1    11156
1  2007Q2    11085
2  2007Q3    10996
3  2007Q4    10798
4  2008Q1    11072


In [129]:
def pull_compustat_quarterly(db, start="1990-01-01", end="2025-12-31"):
    sql = f"""
        SELECT
            gvkey,
            datadate::date,
            fyearq, fqtr, fyr,
            indfmt, datafmt, popsrc, consol,
            curcdq, tic, cusip, cik,

            -- raw variables
            prccq,      -- price (quarter-end)
            cshoq,      -- common shares outstanding (millions)
            ibq,        -- income before extraordinary items (quarter)
            atq,        -- total assets (quarter)
            ltq,        -- total liabilities (quarter)
            seqq,       -- stockholders' equity (quarter)
            ceqq,       -- common equity (quarter)
            pstkq,      -- preferred stock (quarter)
            txditcq    -- deferred taxes & ITC (quarter)
        FROM comp.fundq
        WHERE datadate BETWEEN DATE '{start}' AND DATE '{end}'
          AND indfmt = 'INDL'
          AND datafmt = 'STD'
          AND popsrc = 'D'
          AND consol = 'C'
    """

    df = db.raw_sql(sql, date_cols=['datadate'])

    # numeric helpers
    prccq = pd.to_numeric(df['prccq'], errors='coerce')
    cshoq = pd.to_numeric(df['cshoq'], errors='coerce')  # in millions
    ibq   = pd.to_numeric(df['ibq'],   errors='coerce')
    atq   = pd.to_numeric(df['atq'],   errors='coerce')
    ltq   = pd.to_numeric(df['ltq'],   errors='coerce')
    seqq  = pd.to_numeric(df['seqq'],  errors='coerce')
    ceqq  = pd.to_numeric(df['ceqq'],  errors='coerce')
    pstkq = pd.to_numeric(df['pstkq'], errors='coerce')
    txd   = pd.to_numeric(df['txditcq'], errors='coerce')

    # derived metrics
    df['mktcap'] = prccq * cshoq                     # $ * (millions of shares) -> $ millions
    df['roa_q']  = ibq / atq
    df['lev_q']  = ltq / atq

    be_from_seqq = seqq + txd - pstkq
    be_from_ceqq = ceqq + txd - pstkq
    df['be_q']   = be_from_seqq.where(seqq.notna(), be_from_ceqq)

    df['bm_q'] = df['be_q'] / df['mktcap']

    cols = [
        'gvkey','datadate','fyearq','fqtr','fyr','curcdq','tic','cusip','cik',
        'prccq','cshoq','mktcap',
        'ibq','atq','ltq','seqq','ceqq','pstkq','txditcq',
        'roa_q','lev_q','be_q','bm_q'
    ]
    return df[cols].sort_values(['gvkey','datadate']).reset_index(drop=True)

# Example:
df_q = pull_compustat_quarterly(db, start="2007-01-01", end="2025-12-31")
df_q

,gvkey,datadate,fyearq,fqtr,fyr,curcdq,tic,cusip,cik,prccq,...,atq,ltq,seqq,ceqq,pstkq,txditcq,roa_q,lev_q,be_q,bm_q
0,001004,2007-02-28,2006,3,5,USD,AIR,000361105,0000001750,29.08,...,1010.849,541.148,469.701,469.701,0.0,37.637,0.015352,0.53534,507.338,0.468633
1,001004,2007-05-31,2006,4,5,USD,AIR,000361105,0000001750,32.5,...,1067.633,573.39,494.243,494.243,0.0,40.121,0.016595,0.537067,534.364,0.435791
2,001004,2007-08-31,2007,1,5,USD,AIR,000361105,0000001750,31.4,...,1076.976,567.596,509.38,509.38,0.0,40.471,0.014165,0.527028,549.851,0.46434
3,001004,2007-11-30,2007,2,5,USD,AIR,000361105,0000001750,33.02,...,1137.29,606.963,530.327,530.327,0.0,41.573,0.015729,0.533692,571.9,0.459131
4,001004,2008-02-29,2007,3,5,USD,AIR,000361105,0000001750,25.89,...,1333.454,768.958,564.496,564.496,0.0,19.426,0.015212,0.576666,583.922,0.582218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
868911,366911,2024-06-30,2024,2,12,USD,AMRZ,H2927K103,0002035989,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
868912,366911,2024-09-30,2024,3,12,USD,AMRZ,H2927K103,0002035989,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
868913,366911,2024-12-31,2024,4,12,USD,AMRZ,H2927K103,0002035989,<NA>,...,23805.0,13891.0,9915.0,9915.0,0.0,936.0,0.035455,0.583533,10851.0,<NA>
868914,366911,2025-03-31,2025,1,12,USD,AMRZ,H2927K103,0002035989,<NA>,...,23194.0,13444.0,9750.0,9750.0,0.0,937.0,-0.003751,0.579633,10687.0,<NA>


In [137]:
df_q['YearQuarter'] = pd.to_datetime(df_q['datadate']).dt.to_period('Q').astype(str)
reprisk_merged_yearly['gvkey'] = pd.to_numeric(reprisk_merged_yearly['gvkey'], errors='coerce').astype('Int64')
df_q['gvkey'] = pd.to_numeric(df_q['gvkey'], errors='coerce').astype('Int64')
df_q.drop_duplicates(['gvkey','YearQuarter'], keep='last', inplace=True)
reprisk_with_metrics = reprisk_merged_yearly.merge(df_q[['gvkey','YearQuarter','mktcap','bm_q','lev_q','roa_q']], left_on=['gvkey','YearQuarter_prior'],right_on=['gvkey', 'YearQuarter'], how='left')
reprisk_with_metrics.isna().sum()

gvkey                                 0
YearMonth                             0
n_material                            0
n_nonmaterial                         0
n_car_1_material                      0
n_car_5_material                      0
n_reach_material                      0
n_severity_material                   0
cusip                                 0
SICS Codified Industry                0
Codified SICS Sector                  0
year                                  0
industry_n_material                   0
industry_n_car_1_material             0
industry_n_car_5_material             0
industry_n_reach_material             0
industry_n_severity_material          0
date                                  0
mktrf                                 0
smb                                   0
hml                                   0
rf                                    0
umd                                   0
YearQuarter_x                         0
YearQuarter_prior                     0


In [149]:
reprisk_with_metrics.groupby('YearQuarter_prior')['gvkey'].nunique()

YearQuarter_prior
2006Q4    11718
2007Q1    11718
2007Q2    11718
2007Q3    11718
2007Q4    11718
          ...  
2022Q3    11718
2022Q4    11718
2023Q1    11718
2023Q2    11718
2023Q3    11718
Name: gvkey, Length: 68, dtype: int64

In [150]:
not_na = reprisk_with_metrics.dropna(subset=['roa_q'])
print(not_na['gvkey'].nunique())
not_na.groupby('YearQuarter_prior')['gvkey'].nunique()

9830


YearQuarter_prior
2007Q1    5566
2007Q2    5606
2007Q3    5514
2007Q4    5627
2008Q1    5468
          ... 
2022Q3    4586
2022Q4    4630
2023Q1    4487
2023Q2    4496
2023Q3    4360
Name: gvkey, Length: 67, dtype: int64

In [636]:
# ---------- Institutional Ownership ----------

# Export unique cusips to txt
reprisk_merged_yearly['cusip'].dropna().astype(str).str.replace(r'\.0$', '', regex=True).drop_duplicates().to_csv('data/cusips.txt', index=False, header=False)

In [827]:
ownership_df = pd.read_csv('data/inst_ownership.csv')  # Exported from LSEG Stock Ownership Tool
ownership_df = ownership_df[['cusip', 'rdate', 'InstOwn_Perc']].sort_values(['cusip', 'rdate'])
ownership_df['year'] = pd.to_datetime(ownership_df['rdate']).dt.year + 1
ownership_df.drop_duplicates(['cusip', 'year'], keep='last', inplace=True)
ownership_df

,cusip,rdate,InstOwn_Perc,year
62148,00030710,2014-12-31,2.646183e-01,2015
70952,00030710,2015-12-31,4.817053e-01,2016
79872,00030710,2016-12-31,4.499213e-01,2017
88711,00030710,2017-12-31,6.102770e-01,2018
98929,00030710,2018-12-31,4.897175e-01,2019
...,...,...,...,...
104123,Y1477R20,2019-03-31,4.851384e-06,2020
114648,Y1477R20,2020-03-31,4.481560e-05,2021
146015,Y1477R20,2023-03-31,3.830040e-08,2024
112048,Y1501010,2019-12-31,4.962090e-05,2020


In [828]:
outs = [
    size_df, btm_df, leverage_df, rd_df, ad_df, roa_df
]

df = reprisk_merged_yearly.copy()

for o in outs:
    o['year'] = pd.to_numeric(o['year'], errors='coerce').astype('Int64')
    o['gvkey'] = pd.to_numeric(o['gvkey'], errors='coerce').astype('Int64')
    df = df.merge(o, on=['gvkey','year'], how='left')

df =df.merge(ownership_df, left_on=['cusip','year'], right_on=['cusip','year'], how='left')
df

,gvkey,YearMonth,n_material,n_nonmaterial,n_car_1_material,n_car_5_material,n_reach_material,n_severity_material,cusip,SICS Codified Industry,...,industry_n_reach_material,industry_n_severity_material,size,btm,leverage,rd_intensity,ad_intensity,roa,rdate,InstOwn_Perc
0,1004.0,2007-01,0,0,0,0,0,0,00036110,Industrial Machinery & Goods,...,5.0,5.0,6.258968,0.602174,0.570157,<NA>,<NA>,0.041101,NaN,NaN
1,1004.0,2007-02,0,0,0,0,0,0,00036110,Industrial Machinery & Goods,...,7.0,7.0,6.258968,0.602174,0.570157,<NA>,<NA>,0.041101,NaN,NaN
2,1004.0,2007-03,0,0,0,0,0,0,00036110,Industrial Machinery & Goods,...,6.0,6.0,6.258968,0.602174,0.570157,<NA>,<NA>,0.041101,NaN,NaN
3,1004.0,2007-04,0,0,0,0,0,0,00036110,Industrial Machinery & Goods,...,7.0,7.0,6.258968,0.602174,0.570157,<NA>,<NA>,0.041101,NaN,NaN
4,1004.0,2007-05,0,0,0,0,0,0,00036110,Industrial Machinery & Goods,...,8.0,9.0,6.258968,0.602174,0.570157,<NA>,<NA>,0.041101,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3283171,367430.0,2023-08,0,0,0,0,0,0,NaN,Agricultural Products,...,5.0,5.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN
3283172,367430.0,2023-09,0,0,0,0,0,0,NaN,Agricultural Products,...,13.0,13.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN
3283173,367430.0,2023-10,0,0,0,0,0,0,NaN,Agricultural Products,...,10.0,8.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN
3283174,367430.0,2023-11,0,0,0,0,0,0,NaN,Agricultural Products,...,14.0,12.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN


# 3. Calculate Returns

In [314]:
project_root = Path(r"E:\GermanBusinessPanelTeam\Schrader\Forschung\ESGmateriality")

# ── Load WRDS creds from wrds.env in the project root ─────────────────────────
env_path = project_root / "wrds.env"
load_dotenv(dotenv_path=env_path)
# ── Connect to WRDS ───────────────────────────────────────────────────────────
db = wrds.Connection(
    wrds_username=os.getenv("WRDS_YALE_USERNAME"),
    wrds_password=os.getenv("WRDS_YALE_PASSWORD")
)

Loading library list...
Done


In [96]:
# ---------- PARAMETERS ----------
# set your window
START = "2005-01-01"
END   = "2024-12-31"
from pandas.tseries.offsets import MonthEnd

In [97]:

# ---------- 1) CRSP monthly returns (common stocks) ----------
# ret = total return; prc, shrout used for market equity; we’ll add delist return
sql_msf = f"""
select permno, date, ret, retx, prc, shrout
from crsp.msf
where date between '{START}' and '{END}'
"""
msf = db.raw_sql(sql_msf, date_cols=['date'])
msf['me'] = msf['prc'].abs() * msf['shrout'] * 1000.0

In [98]:
# ---- 2) Names table to get shrcd/exchcd as of date ----
# Use msenames (monthly security names history). Filter to primary share classes (optional).
sql_names = """
select permno, namedt, coalesce(nameendt, '9999-12-31') as nameendt, shrcd, exchcd
from crsp.msenames
"""
names = db.raw_sql(sql_names, date_cols=['namedt', 'nameendt'])

# Merge names onto msf by permno where date is within [namedt, nameendt]
msf = msf.merge(names, on='permno', how='left')
msf = msf[(msf['date'] >= msf['namedt']) & (msf['date'] <= msf['nameendt'])].copy()

# Optional: keep common shares on major exchanges
msf = msf[msf['shrcd'].isin([10, 11]) & msf['exchcd'].isin([1, 2, 3])]


In [99]:
# ---- 3) Add delisting returns and compute adjusted return ----
sql_dl = f"""
select permno, dlstdt as date, dlret
from crsp.msedelist
where dlstdt between '{START}' and '{END}'
"""
dl = db.raw_sql(sql_dl, date_cols=['date'])

# Merge dlret onto the monthly file and build adjusted returns:
# ret_adj = (1 + ret) * (1 + dlret) - 1  (with dlret=0 if missing)
crsp = msf.merge(dl, on=['permno','date'], how='left')
crsp['ret_adj'] = (1.0 + crsp['ret'].astype(float)) * (1.0 + crsp['dlret'].fillna(0).astype(float)) - 1.0


In [100]:
# ---- 4) Lagged ME for VW weights (use ME at t-1 for month t) ----
crsp = crsp.sort_values(['permno','date'])
crsp['me_lag'] = crsp.groupby('permno')['me'].shift(1)

crsp_panel = crsp[['permno','date','ret_adj','me','me_lag','shrcd','exchcd']].copy()
crsp_panel

,permno,date,ret_adj,me,me_lag,shrcd,exchcd
0,10001,2005-01-31,-0.040580,17178900.0,<NA>,11,3
4808,10001,2005-02-28,-0.045166,16428279.0,17178900.0,11,3
9621,10001,2005-03-31,0.124822,18663750.0,16428279.0,11,3
14424,10001,2005-04-29,-0.074684,17269875.0,18663750.0,11,3
19223,10001,2005-05-31,0.219030,21052500.0,17269875.0,11,3
...,...,...,...,...,...,...,...
946608,93436,2024-08-30,-0.077391,684004370400.000122,741380136746.400024,11,3
950467,93436,2024-09-30,0.221942,839047410000.0,684004370400.000122,11,3
954312,93436,2024-10-31,-0.045025,802033523100.599976,839047410000.0,11,3
958141,93436,2024-11-29,0.381469,1107984309600.000244,802033523100.599976,11,3


In [101]:
# ---- 5) Fama–French monthly factors ----
ff = db.raw_sql(f"""
select *
from ff.factors_monthly
where date between '{START}' and '{END}'
""", date_cols=['date'])

# Convert to decimals if table is in %
for col in ff.columns:
    if col.lower() != 'date' and pd.api.types.is_numeric_dtype(ff[col]):
        med = ff[col].abs().median(skipna=True)
        if pd.notna(med) and med > 0.5:  # crude but effective
            ff[col] = ff[col] / 100.0

ff = ff.rename(columns={
    'mktrf':'MKT_RF', 'smb':'SMB', 'hml':'HML', 'rf':'RF',
    'rmw':'RMW' if 'rmw' in ff.columns else 'RMW',
    'cma':'CMA' if 'cma' in ff.columns else 'CMA',
    'umd':'MOM' if 'umd' in ff.columns else 'MOM'
})
ff['date'] = ff['date'] + MonthEnd(0)
ff

,date,MKT_RF,SMB,HML,RF,year,month,MOM,dateff
0,2005-01-31,-0.0275,-0.0166,0.0206,0.0016,20.05,0.01,0.0296,2005-01-31
1,2005-02-28,0.0188,-0.0057,0.0141,0.0016,20.05,0.02,0.0343,2005-02-28
2,2005-03-31,-0.0194,-0.0141,0.0207,0.0021,20.05,0.03,0.0043,2005-03-31
3,2005-04-30,-0.0261,-0.0393,0.0005,0.0021,20.05,0.04,-0.0068,2005-04-29
4,2005-05-31,0.0365,0.0286,-0.0058,0.0024,20.05,0.05,0.0037,2005-05-31
...,...,...,...,...,...,...,...,...,...
235,2024-08-31,0.016,-0.0349,-0.011,0.0048,20.24,0.08,0.0481,2024-08-30
236,2024-09-30,0.0172,-0.0013,-0.0277,0.004,20.24,0.09,-0.0062,2024-09-30
237,2024-10-31,-0.01,-0.0099,0.0086,0.0039,20.24,0.1,0.03,2024-10-31
238,2024-11-30,0.0649,0.0446,0.0015,0.004,20.24,0.11,0.01,2024-11-29


In [102]:
# --- pull CUSIP/NCUSIP name history ---
sql_names = """
select
    permno,
    namedt,
    coalesce(nameendt, '9999-12-31') as nameendt,
    cusip,
    ncusip
from crsp.msenames
"""
names = db.raw_sql(sql_names, date_cols=['namedt','nameendt'])

In [103]:
names.drop_duplicates(['permno'], keep='last', inplace=True)

In [104]:
# --- align to month-end dates in crsp_panel ---
# merge then filter rows where crsp date is within [namedt, nameendt]
crsp_with_cusip = crsp_panel.merge(names, on='permno', how='left')

# 4. Merge all

In [105]:
#----------------------------
# 0) Helpers
# ----------------------------

def choose_primary_class(df, by=['cusip', 'date']):
    """
    If multiple PERMNOs share the same cusip8 at a date,
    keep the one with the largest lagged market cap (me_lag).
    """
    df = df.copy()
    df['_rank'] = df.groupby(by)['me_lag'].rank(method='first', ascending=False)
    out = df.loc[df['_rank'] == 1].drop(columns=['_rank'])
    return out


# ----------------------------
# 1) Prepare CRSP monthly panel
# ----------------------------
crsp = crsp_with_cusip.copy()
crsp['date'] = pd.to_datetime(crsp['date'])

# If same cusip8 maps to multiple permnos per month, keep largest me_lag
crsp_1class = choose_primary_class(crsp, by=['cusip', 'date'])
# ----------------------------
# 2) Prepare factors
# ----------------------------
ff = ff.copy()
ff['date'] = pd.to_datetime(ff['date']) + MonthEnd(0)

In [106]:
# For industry data
crsp_with_cusip['month'] = pd.to_datetime(crsp_with_cusip['date']).dt.to_period('M')
crsp_with_cusip['month_prior'] = pd.to_datetime(crsp_with_cusip['date']).dt.to_period('M')-1

In [107]:
reprisk_with_metrics['YearMonth'] = pd.to_datetime(reprisk_with_metrics['YearMonth']).dt.to_period('M')

TypeError: Passing PeriodDtype data is invalid. Use `data.to_timestamp()` instead

In [114]:
reprisk_with_metrics["cusip"] = reprisk_with_metrics["cusip"].astype(str).str.strip().str.upper().str.slice(0, 8)

panel_firms = crsp_with_cusip[['permno', 'date', 'ret_adj', 'me', 'me_lag', 'cusip', 'month', 'month_prior']].merge(
    reprisk_with_metrics[['cusip', 'gvkey', 'year', 'Codified SICS Sector ', 'SICS Codified Industry ', 'YearMonth',
        'size','roa_q','lev_q','bm_q', 'n_material', 'n_car_1_material','n_car_5_material','n_reach_material','n_severity_material', 'industry_n_car_1_material', 'industry_n_car_5_material',
        'industry_n_material', 'industry_n_reach_material', 'industry_n_severity_material']], left_on=['cusip', 'month_prior'], right_on=['cusip', 'YearMonth'], how='left')
ff['dateff'] = pd.to_datetime(ff['dateff'])
panel_firms = panel_firms.merge(ff[['dateff', 'date', 'MKT_RF', 'SMB', 'HML', 'RF', 'MOM']], left_on='date',
                                right_on='dateff', how='left', suffixes=('', '_ff'))
panel_firms

,permno,date,ret_adj,me,me_lag,cusip,month,month_prior,gvkey,year,...,industry_n_material,industry_n_reach_material,industry_n_severity_material,dateff,date_ff,MKT_RF,SMB,HML,RF,MOM
0,10001,2005-01-31,-0.040580,17178900.0,<NA>,36720410,2005-01,2004-12,NaN,NaN,...,NaN,NaN,NaN,2005-01-31,2005-01-31,-0.0275,-0.0166,0.0206,0.0016,0.0296
1,10001,2005-02-28,-0.045166,16428279.0,17178900.0,36720410,2005-02,2005-01,NaN,NaN,...,NaN,NaN,NaN,2005-02-28,2005-02-28,0.0188,-0.0057,0.0141,0.0016,0.0343
2,10001,2005-03-31,0.124822,18663750.0,16428279.0,36720410,2005-03,2005-02,NaN,NaN,...,NaN,NaN,NaN,2005-03-31,2005-03-31,-0.0194,-0.0141,0.0207,0.0021,0.0043
3,10001,2005-04-29,-0.074684,17269875.0,18663750.0,36720410,2005-04,2005-03,NaN,NaN,...,NaN,NaN,NaN,2005-04-29,2005-04-30,-0.0261,-0.0393,0.0005,0.0021,-0.0068
4,10001,2005-05-31,0.219030,21052500.0,17269875.0,36720410,2005-05,2005-04,NaN,NaN,...,NaN,NaN,NaN,2005-05-31,2005-05-31,0.0365,0.0286,-0.0058,0.0024,0.0037
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
961941,93436,2024-08-30,-0.077391,684004370400.000122,741380136746.400024,88160R10,2024-08,2024-07,NaN,NaN,...,NaN,NaN,NaN,2024-08-30,2024-08-31,0.016,-0.0349,-0.011,0.0048,0.0481
961942,93436,2024-09-30,0.221942,839047410000.0,684004370400.000122,88160R10,2024-09,2024-08,NaN,NaN,...,NaN,NaN,NaN,2024-09-30,2024-09-30,0.0172,-0.0013,-0.0277,0.004,-0.0062
961943,93436,2024-10-31,-0.045025,802033523100.599976,839047410000.0,88160R10,2024-10,2024-09,NaN,NaN,...,NaN,NaN,NaN,2024-10-31,2024-10-31,-0.01,-0.0099,0.0086,0.0039,0.03
961944,93436,2024-11-29,0.381469,1107984309600.000244,802033523100.599976,88160R10,2024-11,2024-10,NaN,NaN,...,NaN,NaN,NaN,2024-11-29,2024-11-30,0.0649,0.0446,0.0015,0.004,0.01


In [115]:
# filter for cusip in df
# fill to nine digits
panel_firms = panel_firms[panel_firms['cusip'].isin(reprisk_with_metrics['cusip'].dropna().unique())]
panel_firms

,permno,date,ret_adj,me,me_lag,cusip,month,month_prior,gvkey,year,...,industry_n_material,industry_n_reach_material,industry_n_severity_material,dateff,date_ff,MKT_RF,SMB,HML,RF,MOM
151,10002,2005-01-31,-0.132466,235298350.0,<NA>,05978R10,2005-01,2004-12,NaN,NaN,...,NaN,NaN,NaN,2005-01-31,2005-01-31,-0.0275,-0.0166,0.0206,0.0016,0.0296
152,10002,2005-02-28,-0.033255,227473440.0,235298350.0,05978R10,2005-02,2005-01,NaN,NaN,...,NaN,NaN,NaN,2005-02-28,2005-02-28,0.0188,-0.0057,0.0141,0.0016,0.0343
153,10002,2005-03-31,-0.013081,225109280.0,227473440.0,05978R10,2005-03,2005-02,NaN,NaN,...,NaN,NaN,NaN,2005-03-31,2005-03-31,-0.0194,-0.0141,0.0207,0.0021,0.0043
154,10002,2005-04-29,-0.066700,209263420.0,225109280.0,05978R10,2005-04,2005-03,NaN,NaN,...,NaN,NaN,NaN,2005-04-29,2005-04-30,-0.0261,-0.0393,0.0005,0.0021,-0.0068
155,10002,2005-05-31,0.046586,218932980.0,209263420.0,05978R10,2005-05,2005-04,NaN,NaN,...,NaN,NaN,NaN,2005-05-31,2005-05-31,0.0365,0.0286,-0.0058,0.0024,0.0037
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
961941,93436,2024-08-30,-0.077391,684004370400.000122,741380136746.400024,88160R10,2024-08,2024-07,NaN,NaN,...,NaN,NaN,NaN,2024-08-30,2024-08-31,0.016,-0.0349,-0.011,0.0048,0.0481
961942,93436,2024-09-30,0.221942,839047410000.0,684004370400.000122,88160R10,2024-09,2024-08,NaN,NaN,...,NaN,NaN,NaN,2024-09-30,2024-09-30,0.0172,-0.0013,-0.0277,0.004,-0.0062
961943,93436,2024-10-31,-0.045025,802033523100.599976,839047410000.0,88160R10,2024-10,2024-09,NaN,NaN,...,NaN,NaN,NaN,2024-10-31,2024-10-31,-0.01,-0.0099,0.0086,0.0039,0.03
961944,93436,2024-11-29,0.381469,1107984309600.000244,802033523100.599976,88160R10,2024-11,2024-10,NaN,NaN,...,NaN,NaN,NaN,2024-11-29,2024-11-30,0.0649,0.0446,0.0015,0.004,0.01


In [116]:
panel_firms.to_csv('Output/panel_industries.csv', index=False)

# 5. Portfolios based on sum

In [337]:
df = pd.read_csv('Output/panel_industries.csv')

In [338]:
df = df[df['YearMonth']>'2006-12']

In [339]:
cols = ['n_material','n_car_1_material','n_car_5_material','n_reach_material','n_severity_material',
        'industry_n_material','industry_n_car_1_material','industry_n_car_5_material',
        'industry_n_reach_material','industry_n_severity_material']

df = df.sort_values(['permno','date'])
for c in cols:
    df[f'{c}_24m'] = df.groupby('permno')[c].transform(lambda s: s.fillna(0).rolling(24, min_periods=1).sum())

df

,permno,date,ret_adj,me,me_lag,cusip,month,month_prior,gvkey,year,...,n_material_24m,n_car_1_material_24m,n_car_5_material_24m,n_reach_material_24m,n_severity_material_24m,industry_n_material_24m,industry_n_car_1_material_24m,industry_n_car_5_material_24m,industry_n_reach_material_24m,industry_n_severity_material_24m
25,10002,2007-02-28,0.033054,2.477735e+08,2.398457e+08,05978R10,2007-02,2007-01,19049.0,2007.0,...,0.0,0.0,0.0,0.0,0.0,7.0,5.0,5.0,9.0,9.0
26,10002,2007-03-30,-0.040559,2.366534e+08,2.477735e+08,05978R10,2007-03,2007-02,19049.0,2007.0,...,0.0,0.0,0.0,0.0,0.0,18.0,6.0,6.0,20.0,20.0
27,10002,2007-04-30,-0.053403,2.240155e+08,2.366534e+08,05978R10,2007-04,2007-03,19049.0,2007.0,...,0.0,0.0,0.0,0.0,0.0,23.0,9.0,9.0,25.0,25.0
28,10002,2007-05-31,0.000499,2.241875e+08,2.240155e+08,05978R10,2007-05,2007-04,19049.0,2007.0,...,0.0,0.0,0.0,0.0,0.0,30.0,11.0,10.0,28.0,28.0
29,10002,2007-06-29,0.054391,2.351160e+08,2.241875e+08,05978R10,2007-06,2007-05,19049.0,2007.0,...,0.0,0.0,0.0,0.0,0.0,70.0,11.0,14.0,71.0,71.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
812611,93436,2023-09-29,-0.030456,7.954494e+11,8.191443e+11,88160R10,2023-09,2023-08,184996.0,2023.0,...,42.0,145.0,121.0,150.0,154.0,555.0,2063.0,1449.0,2089.0,2064.0
812612,93436,2023-10-31,-0.197346,6.384545e+11,7.954494e+11,88160R10,2023-10,2023-09,184996.0,2023.0,...,43.0,147.0,122.0,149.0,153.0,546.0,2143.0,1431.0,2133.0,2108.0
812613,93436,2023-11-30,0.195379,7.631954e+11,6.384545e+11,88160R10,2023-11,2023-10,184996.0,2023.0,...,46.0,154.0,129.0,154.0,158.0,557.0,2216.0,1550.0,2216.0,2191.0
812614,93436,2023-12-29,0.034988,7.914088e+11,7.631954e+11,88160R10,2023-12,2023-11,184996.0,2023.0,...,49.0,156.0,131.0,156.0,160.0,554.0,2260.0,1594.0,2260.0,2235.0


In [340]:
df.dropna(subset=['ret_adj'], inplace=True)

In [341]:
def add_portfolios_three_selections(
    df: pd.DataFrame,
    date_col: str = "YearMonth",
    industry_col: str = "SICS Codified Industry",
):
    """
    Adds tertile/quintile bins + top/bottom flags for three scopes:
      (A) firm incidents within month×industry  → *_tertile_industry, *_quintile_industry, flags
      (B) firm incidents within month (overall) → *_tertile, *_quintile, flags
      (C) industry-peers incidents within month → *_tertile_industry_peers, *_quintile_industry_peers, flags

    Firm incident columns:
      ['n_material','n_car_1_material','n_car_5_material','n_reach_material','n_severity_material']

    Industry-peers incident columns:
      ['industry_n_material','industry_n_car_1_material','industry_n_car_5_material',
       'industry_n_reach_material','industry_n_severity_material']
    """
    out = df.copy()

    # --- Month key (safe parsing) ---
    if date_col not in out.columns:
        raise KeyError(f"{date_col} not found.")
    s = out[date_col]
    if isinstance(s.dtype, pd.PeriodDtype):
        out["_mkey_"] = s.asfreq("M")
    elif pd.api.types.is_datetime64_any_dtype(s):
        out["_mkey_"] = s.dt.to_period("M")
    else:
        out["_mkey_"] = pd.to_datetime(s, errors="coerce").dt.to_period("M")

    # --- Column sets (keep only numeric present in df) ---
    firm_cols = [
        'n_material','n_car_1_material','n_car_5_material','n_reach_material','n_severity_material'
    ]
    firm_cols = [c for c in firm_cols if c in out.columns and pd.api.types.is_numeric_dtype(out[c])]

    ind_cols = [
        'industry_n_material','industry_n_car_1_material','industry_n_car_5_material',
        'industry_n_reach_material','industry_n_severity_material'
    ]
    ind_cols = [c for c in ind_cols if c in out.columns and pd.api.types.is_numeric_dtype(out[c])]

    def _bins_by_quantiles(s: pd.Series, quantiles) -> pd.Series:
        """Return Int64 bins using cutpoints from s.quantile(quantiles)."""
        s = pd.to_numeric(s, errors="coerce")
        mask = s.notna()
        res = pd.Series(pd.NA, index=s.index, dtype="Int64")
        n = int(mask.sum())
        if n == 0:
            return res
        nbins = len(quantiles) + 1
        if n < nbins:
            return res
        x = s[mask]
        if x.min() == x.max():
            res.loc[mask] = (nbins + 1) // 2   # middle bin (2 for tertiles, 3 for quintiles)
            return res
        qs = x.quantile(quantiles)
        if nbins == 3:  # tertiles
            q1, q2 = qs.iloc[0], qs.iloc[1]
            b = pd.Series(2, index=x.index, dtype="Int64")
            b[x <= q1] = 1
            b[x >  q2] = 3
        else:          # quintiles
            q20, q40, q60, q80 = qs.iloc[0], qs.iloc[1], qs.iloc[2], qs.iloc[3]
            b = pd.Series(3, index=x.index, dtype="Int64")
            b[x <= q20] = 1
            b[(x > q20) & (x <= q40)] = 2
            b[(x > q40) & (x <= q60)] = 3
            b[(x > q60) & (x <= q80)] = 4
            b[x > q80] = 5
        res.loc[b.index] = b
        return res

    def _apply_for_scope(cols, group_keys, suffix: str):
        g = out.groupby(group_keys, observed=True, sort=False)
        for col in cols:
            # tertiles
            tert = g[col].transform(lambda s: _bins_by_quantiles(s, [1/3, 2/3]))
            out[f"{col}_tertile{suffix}"]          = tert
            out[f"{col}_top_tertile{suffix}"]      = (tert == 3).astype("Int8")
            out[f"{col}_bottom_tertile{suffix}"]   = (tert == 1).astype("Int8")

            # quintiles (always computed)
            quin = g[col].transform(lambda s: _bins_by_quantiles(s, [0.2, 0.4, 0.6, 0.8]))
            out[f"{col}_quintile{suffix}"]         = quin
            out[f"{col}_top_quintile{suffix}"]     = (quin == 5).astype("Int8")
            out[f"{col}_bottom_quintile{suffix}"]  = (quin == 1).astype("Int8")

    # (A) Within month × industry: firm incidents
    if firm_cols and industry_col in out.columns:
        _apply_for_scope(firm_cols, ["_mkey_", industry_col], suffix="_industry")

    # (B) Within month overall: firm incidents
    if firm_cols:
        _apply_for_scope(firm_cols, ["_mkey_"], suffix="")

    # (C) Within month overall: industry-peers incidents
    if ind_cols:
        _apply_for_scope(ind_cols, ["_mkey_"], suffix="_industry_peers")

    return out.drop(columns=["_mkey_"])


In [302]:
df_ports = add_portfolios_three_selections(
    df,
    date_col="YearMonth",
    industry_col="SICS Codified Industry "
)
df_ports

,permno,date,ret_adj,me,me_lag,cusip,month,month_prior,gvkey,year,...,industry_n_reach_material_bottom_tertile_industry_peers,industry_n_reach_material_quintile_industry_peers,industry_n_reach_material_top_quintile_industry_peers,industry_n_reach_material_bottom_quintile_industry_peers,industry_n_severity_material_tertile_industry_peers,industry_n_severity_material_top_tertile_industry_peers,industry_n_severity_material_bottom_tertile_industry_peers,industry_n_severity_material_quintile_industry_peers,industry_n_severity_material_top_quintile_industry_peers,industry_n_severity_material_bottom_quintile_industry_peers
25,10002,2007-02-28,0.033054,2.477735e+08,2.398457e+08,05978R10,2007-02,2007-01,19049.0,2007.0,...,0,5,1,0,3,1,0,5,1,0
26,10002,2007-03-30,-0.040559,2.366534e+08,2.477735e+08,05978R10,2007-03,2007-02,19049.0,2007.0,...,0,5,1,0,3,1,0,5,1,0
27,10002,2007-04-30,-0.053403,2.240155e+08,2.366534e+08,05978R10,2007-04,2007-03,19049.0,2007.0,...,0,5,1,0,3,1,0,5,1,0
28,10002,2007-05-31,0.000499,2.241875e+08,2.240155e+08,05978R10,2007-05,2007-04,19049.0,2007.0,...,0,4,0,0,3,1,0,4,0,0
29,10002,2007-06-29,0.054391,2.351160e+08,2.241875e+08,05978R10,2007-06,2007-05,19049.0,2007.0,...,0,5,1,0,3,1,0,5,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
812611,93436,2023-09-29,-0.030456,7.954494e+11,8.191443e+11,88160R10,2023-09,2023-08,184996.0,2023.0,...,0,5,1,0,3,1,0,5,1,0
812612,93436,2023-10-31,-0.197346,6.384545e+11,7.954494e+11,88160R10,2023-10,2023-09,184996.0,2023.0,...,0,5,1,0,3,1,0,5,1,0
812613,93436,2023-11-30,0.195379,7.631954e+11,6.384545e+11,88160R10,2023-11,2023-10,184996.0,2023.0,...,0,5,1,0,3,1,0,5,1,0
812614,93436,2023-12-29,0.034988,7.914088e+11,7.631954e+11,88160R10,2023-12,2023-11,184996.0,2023.0,...,0,5,1,0,3,1,0,5,1,0


In [225]:
df_ports.isna().sum()

permno                                                0
date                                                  0
ret_adj                                               0
me                                                    0
me_lag                                              130
                                                   ... 
industry_n_severity_material_24m_top_tertile          0
industry_n_severity_material_24m_bottom_tertile       0
industry_n_severity_material_24m_quintile             0
industry_n_severity_material_24m_top_quintile         0
industry_n_severity_material_24m_bottom_quintile      0
Length: 104, dtype: int64

In [311]:
import re
import pandas as pd
import numpy as np

def portfolio_membership_counts(df, date_col="YearMonth"):
    # month → Period[M]
    if np.issubdtype(df[date_col].dtype, np.datetime64):
        m = df[date_col].dt.to_period("M")
    else:
        m = pd.to_datetime(df[date_col].astype(str), errors="coerce").dt.to_period("M")
    x = df.copy()
    x["_m_"] = m

    # Matches:
    #   <signal>_(top|bottom)_(quintile|tertile)           -> overall
    #   <signal>_(top|bottom)_(quintile|tertile)_industry  -> industry
    #   <signal>_(top|bottom)_(quintile|tertile)_industry_peers -> industry_peers
    pat = re.compile(
        r"^(?P<signal>.+)_(?P<leg>top|bottom)_(?P<bucket>quintile|tertile)"
        r"(?P<scope>_industry(?:_peers)?|)$"  # empty string => overall
    )

    def scope_name(sfx: str) -> str:
        if sfx == "":
            return "overall"
        if sfx == "_industry":
            return "industry"
        if sfx == "_industry_peers":
            return "industry_peers"
        return "overall"

    rows = []
    for c in x.columns:
        mobj = pat.match(c)
        if not mobj:
            continue
        d = mobj.groupdict()
        signal = d["signal"]
        leg    = d["leg"]
        bucket = d["bucket"]
        scope  = scope_name(d["scope"])

        # membership: accept 1/0 ints, bools, or numeric with NaNs
        col = x[c]
        if col.dtype == bool:
            mask = col
        else:
            mask = pd.to_numeric(col, errors="coerce").fillna(0).astype(int).eq(1)

        cnt = x.loc[mask].groupby("_m_").size()
        rows.append(cnt.rename(f"{signal}__{scope}__{bucket}__{leg}"))

    if not rows:
        return pd.DataFrame()

    return (
        pd.concat(rows, axis=1)
          .fillna(0).astype(int)
          .reset_index()
          .rename(columns={"_m_": "YearMonth"})
    )

# Example:
membership = portfolio_membership_counts(df_ports)
membership

In [312]:
def ls_excess_from_top_bottom_named(
    df: pd.DataFrame,
    date_col: str = "YearMonth",
    ret_col: str  = "ret_adj",
    rf_col: str   = "RF",
    weight_col: str = "me_lag",
):
    x = df.copy()

    # Month key -> Period[M]
    if np.issubdtype(x[date_col].dtype, np.datetime64):
        x["_m_"] = x[date_col].dt.to_period("M")
    else:
        x["_m_"] = pd.to_datetime(x[date_col].astype(str), errors="coerce").dt.to_period("M")

    # Excess return and weights
    x["_rex_"] = (
        pd.to_numeric(x.get(ret_col, np.nan), errors="coerce")
        - pd.to_numeric(x.get(rf_col, 0.0), errors="coerce").fillna(0.0)
    )
    x["_w_"] = pd.to_numeric(x.get(weight_col, np.nan), errors="coerce")

    # Recognize columns with optional scope suffix
    pat = re.compile(
        r"^(?P<signal>.+)_(?P<leg>top|bottom)_(?P<bucket>tertile|quintile)"
        r"(?P<scope>_industry(?:_peers)?)?$"
    )

    def scope_tag(scope: str | None) -> str:
        if not scope:
            return "overall"
        if scope == "_industry":
            return "industry"
        if scope == "_industry_peers":
            return "industry_peers"
        return "overall"

    # Collect top/bottom pairs by (signal, scope, bucket)
    pairs = {}
    for c in x.columns:
        m = pat.match(c)
        if not m:
            continue
        d = m.groupdict()
        key = (d["signal"], scope_tag(d["scope"]), d["bucket"])
        pairs.setdefault(key, {"top": None, "bottom": None})
        pairs[key][d["leg"]] = c

    specs = [
        (sig, scp, bkt, cols["top"], cols["bottom"])
        for (sig, scp, bkt), cols in pairs.items()
        if cols["top"] is not None and cols["bottom"] is not None
    ]
    if not specs:
        raise ValueError("No complete top/bottom pairs found for overall/industry/industry_peers.")

    def _clean_signal(sig: str) -> str:
        s = sig
        s = re.sub(r"^n_", "", s)
        s = s.replace("_material_24m", "").replace("_material", "")
        s = s.replace("_24m", "")
        return s.strip("_")

    # --- SAFE mask converter ---
    def _safe_bool(mask: pd.Series) -> pd.Series:
        # works for bool, int/float (0/1), and object with '0'/'1'
        if mask.dtype == bool:
            return mask.astype("boolean").fillna(False)
        m = pd.to_numeric(mask, errors="coerce")
        return m.eq(1).astype("boolean").fillna(False)

    # EW & VW helpers on *excess* returns
    def _ew(mask: pd.Series) -> pd.Series:
        m = _safe_bool(mask)
        if not m.any():
            return pd.Series(dtype=float)
        return x.loc[m].groupby(x.loc[m, "_m_"])["_rex_"].mean()

    def _vw(mask: pd.Series) -> pd.Series:
        m = _safe_bool(mask)
        tmp = x.loc[m, ["_m_", "_rex_", "_w_"]].copy()
        if tmp.empty:
            return pd.Series(dtype=float)
        tmp["_w_"] = tmp["_w_"].clip(lower=0)
        sumw = tmp.groupby("_m_")["_w_"].sum()
        sumw = sumw.where(sumw > 0)  # NaN when no positive weight
        tmp = tmp.join(sumw.rename("_sumw_"), on="_m_")
        tmp["_contrib_"] = tmp["_rex_"] * (tmp["_w_"] / tmp["_sumw_"])
        return tmp.groupby("_m_")["_contrib_"].sum()

    frames = []
    for sig, scp, bkt, top_col, bot_col in sorted(specs):
        top_m = x[top_col]
        bot_m = x[bot_col]

        ew_top, ew_bot = _ew(top_m), _ew(bot_m)
        vw_top, vw_bot = _vw(top_m), _vw(bot_m)

        # align unions of months
        idx = ew_top.index.union(ew_bot.index)
        ew_ls_ex = ew_top.reindex(idx) - ew_bot.reindex(idx)

        idx_vw = vw_top.index.union(vw_bot.index)
        vw_ls_ex = vw_top.reindex(idx_vw) - vw_bot.reindex(idx_vw)
        vw_ls_ex = vw_ls_ex.reindex(idx)

        base = _clean_signal(sig)
        if scp == "industry":
            base = f"industry_{base}"
        elif scp == "industry_peers":
            base = f"industry_peers_{base}"

        col_ew = f"{base}_{bkt}_ls_ew"
        col_vw = f"{base}_{bkt}_ls_vw"

        frames.append(pd.DataFrame({
            "YearMonth": idx,
            col_ew: ew_ls_ex.values,
            col_vw: vw_ls_ex.values,
        }))

    out = frames[0]
    for f in frames[1:]:
        out = out.merge(f, on="YearMonth", how="outer")

    return out.sort_values("YearMonth").reset_index(drop=True)


In [313]:
df_ls = ls_excess_from_top_bottom_named(
    df_ports,
    date_col="YearMonth",
    ret_col="ret_adj",
    rf_col="RF",
    weight_col="me_lag",
)
df_ls

,YearMonth,industry_peers_industry_n_car_1_quintile_ls_ew,industry_peers_industry_n_car_1_quintile_ls_vw,industry_peers_industry_n_car_1_tertile_ls_ew,industry_peers_industry_n_car_1_tertile_ls_vw,industry_peers_industry_n_car_5_quintile_ls_ew,industry_peers_industry_n_car_5_quintile_ls_vw,industry_peers_industry_n_car_5_tertile_ls_ew,industry_peers_industry_n_car_5_tertile_ls_vw,industry_peers_industry_n_quintile_ls_ew,...,reach_tertile_ls_ew,reach_tertile_ls_vw,industry_severity_quintile_ls_ew,industry_severity_quintile_ls_vw,industry_severity_tertile_ls_ew,industry_severity_tertile_ls_vw,severity_quintile_ls_ew,severity_quintile_ls_vw,severity_tertile_ls_ew,severity_tertile_ls_vw
0,2007-01,-0.007277,-0.004943,-0.007277,-0.004943,-0.009758,-0.004380,-0.007702,-0.004118,-0.005500,...,-0.023924,-0.011437,-0.024144,-0.035310,-0.027852,-0.032447,-0.022697,-0.024004,-0.022697,-0.024004
1,2007-02,0.006250,0.013804,0.006250,0.013804,0.006250,0.013804,0.006250,0.013804,-0.009696,...,0.006067,0.036809,0.001559,0.021858,0.001559,0.021858,0.006565,0.021926,0.006565,0.021926
2,2007-03,-0.031157,-0.005128,-0.031157,-0.005128,-0.036645,-0.025260,-0.036645,-0.025260,-0.039467,...,0.016917,0.011196,0.013431,0.007751,0.013085,0.005132,0.016917,0.011196,0.016917,0.011196
3,2007-04,0.013616,0.042829,-0.025647,-0.010104,-0.024847,-0.009405,-0.024390,-0.007571,-0.007613,...,0.023047,-0.007477,0.009158,-0.024406,0.009158,-0.024406,0.019648,-0.011563,0.019648,-0.011563
4,2007-05,-0.012006,0.009796,-0.010417,0.002787,-0.019502,0.001631,-0.018817,0.001793,-0.014101,...,0.013641,0.011768,0.025424,0.014051,0.025408,0.014035,0.018303,0.014206,0.018303,0.014206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,2023-08,0.007513,-0.000675,0.006169,-0.014052,0.001993,-0.014365,0.002296,-0.019216,0.002113,...,0.004998,0.001438,0.008121,0.000815,0.004500,0.000705,0.001731,-0.001148,0.001731,-0.001148
200,2023-09,0.007605,-0.013586,0.018436,-0.013823,0.015277,-0.001421,-0.018117,-0.014550,0.020745,...,0.024368,0.023411,0.025808,0.026964,0.025872,0.026087,0.028035,0.027538,0.028035,0.027538
201,2023-10,-0.007662,0.009303,0.000764,0.024287,-0.011749,-0.016558,0.004562,-0.004092,0.009119,...,-0.027422,-0.012753,0.001499,-0.006413,-0.000416,-0.008034,-0.021784,-0.010482,-0.021784,-0.010482
202,2023-11,0.005668,0.015046,0.002535,0.016108,0.014657,0.005715,0.012749,0.021855,0.024720,...,-0.031855,-0.026213,-0.040626,-0.018063,-0.040201,-0.018572,-0.040076,-0.027687,-0.040076,-0.027687


In [315]:
# ---- 5) Fama–French monthly factors ----
ff = db.raw_sql(f"""
select *
from ff.factors_monthly
where date between '{START}' and '{END}'
""", date_cols=['date'])

# Convert to decimals if table is in %
for col in ff.columns:
    if col.lower() != 'date' and pd.api.types.is_numeric_dtype(ff[col]):
        med = ff[col].abs().median(skipna=True)
        if pd.notna(med) and med > 0.5:  # crude but effective
            ff[col] = ff[col] / 100.0

ff = ff.rename(columns={
    'mktrf':'MKT_RF', 'smb':'SMB', 'hml':'HML', 'rf':'RF',
    'rmw':'RMW' if 'rmw' in ff.columns else 'RMW',
    'cma':'CMA' if 'cma' in ff.columns else 'CMA',
    'umd':'MOM' if 'umd' in ff.columns else 'MOM'
})
ff['date'] = ff['date'] + MonthEnd(0)
ff['month'] = pd.to_datetime(ff['date']).dt.to_period('M')
ff

,date,MKT_RF,SMB,HML,RF,year,month,MOM,dateff
0,2005-01-31,-0.0275,-0.0166,0.0206,0.0016,20.05,2005-01,0.0296,2005-01-31
1,2005-02-28,0.0188,-0.0057,0.0141,0.0016,20.05,2005-02,0.0343,2005-02-28
2,2005-03-31,-0.0194,-0.0141,0.0207,0.0021,20.05,2005-03,0.0043,2005-03-31
3,2005-04-30,-0.0261,-0.0393,0.0005,0.0021,20.05,2005-04,-0.0068,2005-04-29
4,2005-05-31,0.0365,0.0286,-0.0058,0.0024,20.05,2005-05,0.0037,2005-05-31
...,...,...,...,...,...,...,...,...,...
235,2024-08-31,0.016,-0.0349,-0.011,0.0048,20.24,2024-08,0.0481,2024-08-30
236,2024-09-30,0.0172,-0.0013,-0.0277,0.004,20.24,2024-09,-0.0062,2024-09-30
237,2024-10-31,-0.01,-0.0099,0.0086,0.0039,20.24,2024-10,0.03,2024-10-31
238,2024-11-30,0.0649,0.0446,0.0015,0.004,20.24,2024-11,0.01,2024-11-29


In [316]:
df_port_ff = df_ls.merge(ff[['month','MKT_RF','SMB','HML','RF','MOM']], left_on='YearMonth', right_on='month', how='left')
df_port_ff

,YearMonth,industry_peers_industry_n_car_1_quintile_ls_ew,industry_peers_industry_n_car_1_quintile_ls_vw,industry_peers_industry_n_car_1_tertile_ls_ew,industry_peers_industry_n_car_1_tertile_ls_vw,industry_peers_industry_n_car_5_quintile_ls_ew,industry_peers_industry_n_car_5_quintile_ls_vw,industry_peers_industry_n_car_5_tertile_ls_ew,industry_peers_industry_n_car_5_tertile_ls_vw,industry_peers_industry_n_quintile_ls_ew,...,severity_quintile_ls_ew,severity_quintile_ls_vw,severity_tertile_ls_ew,severity_tertile_ls_vw,month,MKT_RF,SMB,HML,RF,MOM
0,2007-01,-0.007277,-0.004943,-0.007277,-0.004943,-0.009758,-0.004380,-0.007702,-0.004118,-0.005500,...,-0.022697,-0.024004,-0.022697,-0.024004,2007-01,0.0138,0.0009,-0.0068,0.0044,0.0017
1,2007-02,0.006250,0.013804,0.006250,0.013804,0.006250,0.013804,0.006250,0.013804,-0.009696,...,0.006565,0.021926,0.006565,0.021926,2007-02,-0.0196,0.0123,-0.0008,0.0038,-0.0129
2,2007-03,-0.031157,-0.005128,-0.031157,-0.005128,-0.036645,-0.025260,-0.036645,-0.025260,-0.039467,...,0.016917,0.011196,0.016917,0.011196,2007-03,0.0071,0.0009,-0.0088,0.0043,0.0258
3,2007-04,0.013616,0.042829,-0.025647,-0.010104,-0.024847,-0.009405,-0.024390,-0.007571,-0.007613,...,0.019648,-0.011563,0.019648,-0.011563,2007-04,0.0349,-0.0221,-0.0144,0.0044,-0.0016
4,2007-05,-0.012006,0.009796,-0.010417,0.002787,-0.019502,0.001631,-0.018817,0.001793,-0.014101,...,0.018303,0.014206,0.018303,0.014206,2007-05,0.0323,0.0016,-0.0059,0.0041,-0.0033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,2023-08,0.007513,-0.000675,0.006169,-0.014052,0.001993,-0.014365,0.002296,-0.019216,0.002113,...,0.001731,-0.001148,0.001731,-0.001148,2023-08,-0.0236,-0.0314,-0.0115,0.0045,0.0378
200,2023-09,0.007605,-0.013586,0.018436,-0.013823,0.015277,-0.001421,-0.018117,-0.014550,0.020745,...,0.028035,0.027538,0.028035,0.027538,2023-09,-0.0523,-0.0251,0.0149,0.0043,0.0033
201,2023-10,-0.007662,0.009303,0.000764,0.024287,-0.011749,-0.016558,0.004562,-0.004092,0.009119,...,-0.021784,-0.010482,-0.021784,-0.010482,2023-10,-0.0315,-0.0386,0.002,0.0047,0.0167
202,2023-11,0.005668,0.015046,0.002535,0.016108,0.014657,0.005715,0.012749,0.021855,0.024720,...,-0.040076,-0.027687,-0.040076,-0.027687,2023-11,0.0888,-0.001,0.0167,0.0044,0.0256


In [317]:
df_port_ff.to_csv('Output/portfolio_sum.csv', index=False)

# 5. Portfolios based on orthogonalization

In [357]:
df = pd.read_csv('Output/panel_industries.csv')
df = df[df['YearMonth'] > '2006-12']
cols = ['n_material', 'n_car_1_material', 'n_car_5_material', 'n_reach_material', 'n_severity_material',
        'industry_n_material', 'industry_n_car_1_material', 'industry_n_car_5_material',
        'industry_n_reach_material', 'industry_n_severity_material']

df = df.sort_values(['permno', 'date'])
for c in cols:
    df[f'{c}_24m'] = df.groupby('permno')[c].transform(lambda s: s.fillna(0).rolling(24, min_periods=1).sum())

df

,permno,date,ret_adj,me,me_lag,cusip,month,month_prior,gvkey,year,...,n_material_24m,n_car_1_material_24m,n_car_5_material_24m,n_reach_material_24m,n_severity_material_24m,industry_n_material_24m,industry_n_car_1_material_24m,industry_n_car_5_material_24m,industry_n_reach_material_24m,industry_n_severity_material_24m
25,10002,2007-02-28,0.033054,2.477735e+08,2.398457e+08,05978R10,2007-02,2007-01,19049.0,2007.0,...,0.0,0.0,0.0,0.0,0.0,7.0,5.0,5.0,9.0,9.0
26,10002,2007-03-30,-0.040559,2.366534e+08,2.477735e+08,05978R10,2007-03,2007-02,19049.0,2007.0,...,0.0,0.0,0.0,0.0,0.0,18.0,6.0,6.0,20.0,20.0
27,10002,2007-04-30,-0.053403,2.240155e+08,2.366534e+08,05978R10,2007-04,2007-03,19049.0,2007.0,...,0.0,0.0,0.0,0.0,0.0,23.0,9.0,9.0,25.0,25.0
28,10002,2007-05-31,0.000499,2.241875e+08,2.240155e+08,05978R10,2007-05,2007-04,19049.0,2007.0,...,0.0,0.0,0.0,0.0,0.0,30.0,11.0,10.0,28.0,28.0
29,10002,2007-06-29,0.054391,2.351160e+08,2.241875e+08,05978R10,2007-06,2007-05,19049.0,2007.0,...,0.0,0.0,0.0,0.0,0.0,70.0,11.0,14.0,71.0,71.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
812611,93436,2023-09-29,-0.030456,7.954494e+11,8.191443e+11,88160R10,2023-09,2023-08,184996.0,2023.0,...,42.0,145.0,121.0,150.0,154.0,555.0,2063.0,1449.0,2089.0,2064.0
812612,93436,2023-10-31,-0.197346,6.384545e+11,7.954494e+11,88160R10,2023-10,2023-09,184996.0,2023.0,...,43.0,147.0,122.0,149.0,153.0,546.0,2143.0,1431.0,2133.0,2108.0
812613,93436,2023-11-30,0.195379,7.631954e+11,6.384545e+11,88160R10,2023-11,2023-10,184996.0,2023.0,...,46.0,154.0,129.0,154.0,158.0,557.0,2216.0,1550.0,2216.0,2191.0
812614,93436,2023-12-29,0.034988,7.914088e+11,7.631954e+11,88160R10,2023-12,2023-11,184996.0,2023.0,...,49.0,156.0,131.0,156.0,160.0,554.0,2260.0,1594.0,2260.0,2235.0


In [358]:
import statsmodels.api as sm

def _to_month_period(s: pd.Series) -> pd.Series:
    """Coerce to monthly PeriodIndex safely."""
    if isinstance(s.dtype, pd.PeriodDtype):
        try:
            return s.dt.asfreq("M")
        except Exception:
            return s.astype("period[M]")
    return pd.to_datetime(s, errors="coerce").dt.to_period("M")

def crosssec_residuals_month(
    df: pd.DataFrame,
    dep_vars,
    controls,
    industry_col: str = "SICS Codified Industry",
    month_col: str = "YearMonth",
):
    out = df.copy()

    # --- keys & basic checks
    if month_col not in out.columns:
        raise KeyError(f"{month_col} not found.")
    if industry_col not in out.columns:
        raise KeyError(f"Missing industry column: {industry_col}")

    out["_m_"] = _to_month_period(out[month_col])

    # coerce numerics for controls and y's
    for c in controls:
        if c not in out.columns:
            raise KeyError(f"Missing control: {c}")
        out[c] = pd.to_numeric(out[c], errors="coerce")

    for y in dep_vars:
        if y not in out.columns:
            raise KeyError(f"Missing dependent var: {y}")
        out[y] = pd.to_numeric(out[y], errors="coerce")
        out[f"resid_{y}"] = np.nan              # across industries (per month)
        out[f"resid_{y}_industry"] = np.nan     # within industry (per month×industry)

    def _fit_resid(sub: pd.DataFrame, y: str):
        """Return residuals (Series) of y ~ controls on sub; None if underidentified."""
        # drop any NA rows in variables used
        use_cols = [y] + controls
        dat = sub[use_cols].replace([np.inf, -np.inf], np.nan).dropna(how="any")
        if dat.empty:
            return None
        X = sm.add_constant(dat[controls].astype(float), has_constant="add")
        yy = dat[y].astype(float)
        # need more rows than parameters
        if X.shape[0] <= X.shape[1]:
            return None
        try:
            fit = sm.OLS(yy, X).fit()
        except Exception:
            return None
        return (yy - fit.fittedvalues)

    # --- loop by month (across industries)
    for m, idx_m in out.groupby("_m_").groups.items():
        sub_m = out.loc[idx_m]

        for y in dep_vars:
            # across industries (no suffix)
            r = _fit_resid(sub_m, y)
            if r is not None:
                out.loc[r.index, f"resid_{y}"] = r.values

            # within industry (month × industry)
            for ind, idx_mi in sub_m.groupby(industry_col, dropna=False).groups.items():
                sub_mi = out.loc[idx_mi]
                r_i = _fit_resid(sub_mi, y)
                if r_i is not None:
                    out.loc[r_i.index, f"resid_{y}_industry"] = r_i.values

    return out.drop(columns=["_m_"])


In [359]:
#dep_vars = ['n_car_1_material','n_car_5_material','n_reach_material','n_severity_material','n_material']
dep_vars = ['n_car_1_material_24m','n_car_5_material_24m','n_reach_material_24m','n_severity_material_24m','n_material_24m']
controls = ['size','bm_q','lev_q','roa_q']

res_df = crosssec_residuals_month(
    df,                                # <- your firm-month panel
    dep_vars=dep_vars,
    controls=controls,
    industry_col='SICS Codified Industry ',
    month_col='YearMonth'                    # monthly cross-section
)

# sanity check: how many non-missing residuals per dep var?
res_df[[f"resid_{y}" for y in dep_vars]].notna().sum()

resid_n_car_1_material_24m       521699
resid_n_car_5_material_24m       521699
resid_n_reach_material_24m       521699
resid_n_severity_material_24m    521699
resid_n_material_24m             521699
dtype: int64

In [361]:
res_df.dropna(subset=['resid_n_car_1_material_24m'], inplace=True)
res_df

,permno,date,ret_adj,me,me_lag,cusip,month,month_prior,gvkey,year,...,resid_n_car_1_material_24m,resid_n_car_1_material_24m_industry,resid_n_car_5_material_24m,resid_n_car_5_material_24m_industry,resid_n_reach_material_24m,resid_n_reach_material_24m_industry,resid_n_severity_material_24m,resid_n_severity_material_24m_industry,resid_n_material_24m,resid_n_material_24m_industry
129,10025,2007-02-28,-0.028094,3.596595e+08,3.700557e+08,00103110,2007-02,2007-01,11903.0,2007.0,...,-0.001732,0.000000,-0.001565,0.000000,-0.002979,0.000000,-0.003356,0.000000,-0.001063,0.000000
130,10025,2007-03-30,-0.051192,3.420650e+08,3.596595e+08,00103110,2007-03,2007-02,11903.0,2007.0,...,-0.005624,0.000000,-0.004964,0.000000,-0.009022,0.000000,-0.009748,0.000000,-0.004319,0.000000
131,10025,2007-04-30,-0.010233,3.340109e+08,3.420650e+08,00103110,2007-04,2007-03,11903.0,2007.0,...,-0.010389,0.000000,-0.008463,0.000000,-0.016994,0.000000,-0.017716,0.000000,-0.009959,0.000000
132,10025,2007-05-31,0.048872,3.503347e+08,3.340109e+08,00103110,2007-05,2007-04,11903.0,2007.0,...,-0.010259,0.000000,-0.008968,0.000000,-0.016761,0.000000,-0.017620,0.000000,-0.009383,0.000000
133,10025,2007-06-29,0.008289,3.398255e+08,3.503347e+08,00103110,2007-06,2007-05,11903.0,2007.0,...,-0.014911,0.000000,-0.016009,0.000000,-0.024249,0.000000,-0.026290,0.000000,-0.016088,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
812602,93436,2022-12-30,-0.367334,3.897415e+11,6.148143e+11,88160R10,2022-12,2022-11,184996.0,2022.0,...,100.756195,95.157371,81.038833,77.546350,146.838228,139.091561,149.117155,141.671255,26.371605,38.050545
812603,93436,2023-01-31,0.406235,5.480859e+11,3.897415e+11,88160R10,2023-01,2022-12,184996.0,2022.0,...,105.747019,99.567799,86.052041,81.956778,149.726655,141.844352,153.001313,145.336467,27.341142,38.983413
812604,93436,2023-02-28,0.187565,6.508877e+11,5.480859e+11,88160R10,2023-02,2023-01,184996.0,2023.0,...,109.750139,104.052607,89.884777,86.083732,152.655544,144.407095,156.871860,148.615256,30.632707,39.580618
812605,93436,2023-03-31,0.008507,6.575059e+11,6.508877e+11,88160R10,2023-03,2023-02,184996.0,2023.0,...,113.705986,107.480691,89.876662,86.122399,148.660978,140.462940,152.851626,144.672335,30.513657,39.472223


In [362]:
def select_residual_tails(
    df: pd.DataFrame,
    resid_cols=None,                       # if None: all columns starting with 'resid_'
    date_col: str = "YearMonth",
    industry_col: str = "SICS Codified Industry",
    buckets=("decile","quintile","tertile"),
) -> pd.DataFrame:
    """
    Build top/bottom flags from residuals.
    - For residuals like 'resid_y' (across-industry): bucket within month.
    - For residuals like 'resid_y_industry' (within-industry): bucket within month×industry.
    IMPORTANT: 'top' = LOWEST residuals, 'bottom' = HIGHEST residuals (flipped).
    """
    out = df.copy()

    # month key
    if date_col not in out.columns:
        raise KeyError(f"{date_col} not found.")
    if isinstance(out[date_col].dtype, pd.PeriodDtype):
        out["_m_"] = out[date_col].dt.asfreq("M")
    else:
        out["_m_"] = pd.to_datetime(out[date_col], errors="coerce").dt.to_period("M")

    # which residual columns?
    if resid_cols is None:
        resid_cols = [c for c in out.columns if c.startswith("resid_")]
    resid_cols = [c for c in resid_cols if c in out.columns]
    if not resid_cols:
        return out.drop(columns=["_m_"])

    # bucket shares
    share = {"decile":0.10, "quintile":0.20, "tertile":1/3}
    buckets = [b for b in buckets if b in share]
    if not buckets:
        return out.drop(columns=["_m_"])

    # tie-preserving flags (FLIPPED: top = lowest, bottom = highest)
    def _top_flag_low(s: pd.Series, p: float) -> pd.Series:
        x = pd.to_numeric(s, errors="coerce")
        thr = x.quantile(p, interpolation="lower")        # include ties at the low tail
        return x.le(thr) & x.notna()

    def _bottom_flag_high(s: pd.Series, p: float) -> pd.Series:
        x = pd.to_numeric(s, errors="coerce")
        thr = x.quantile(1 - p, interpolation="higher")   # include ties at the high tail
        return x.ge(thr) & x.notna()

    # groupers
    g_month = out.groupby(["_m_"], observed=True, sort=False)
    g_m_ind = (out.groupby(["_m_", industry_col], observed=True, sort=False)
               if industry_col in out.columns else None)

    for col in resid_cols:
        out[col] = pd.to_numeric(out[col], errors="coerce")
        is_within_industry = col.endswith("_industry")

        if is_within_industry:
            if g_m_ind is None:
                raise KeyError(f"Industry column '{industry_col}' missing for within-industry residuals.")
            grouper = g_m_ind
            suf = "_industry"     # keep your existing naming convention
        else:
            grouper = g_month
            suf = ""

        for b in buckets:
            p = share[b]
            # NOTE: flipped assignment here
            out[f"{col}_top_{b}{suf}"]    = grouper[col].transform(lambda s, p=p: _top_flag_low(s, p)).astype("Int8")
            out[f"{col}_bottom_{b}{suf}"] = grouper[col].transform(lambda s, p=p: _bottom_flag_high(s, p)).astype("Int8")

    return out.drop(columns=["_m_"])


In [363]:
df_with_flags = select_residual_tails(
    res_df,
    resid_cols=None,
    date_col="YearMonth",
    industry_col="SICS Codified Industry ",
    buckets=("decile","quintile","tertile")
)


df_with_flags

,permno,date,ret_adj,me,me_lag,cusip,month,month_prior,gvkey,year,...,resid_n_material_24m_top_quintile,resid_n_material_24m_bottom_quintile,resid_n_material_24m_top_tertile,resid_n_material_24m_bottom_tertile,resid_n_material_24m_industry_top_decile_industry,resid_n_material_24m_industry_bottom_decile_industry,resid_n_material_24m_industry_top_quintile_industry,resid_n_material_24m_industry_bottom_quintile_industry,resid_n_material_24m_industry_top_tertile_industry,resid_n_material_24m_industry_bottom_tertile_industry
129,10025,2007-02-28,-0.028094,3.596595e+08,3.700557e+08,00103110,2007-02,2007-01,11903.0,2007.0,...,0,0,0,0,1,1,1,1,1,1
130,10025,2007-03-30,-0.051192,3.420650e+08,3.596595e+08,00103110,2007-03,2007-02,11903.0,2007.0,...,0,0,0,0,1,1,1,1,1,1
131,10025,2007-04-30,-0.010233,3.340109e+08,3.420650e+08,00103110,2007-04,2007-03,11903.0,2007.0,...,0,0,0,0,1,1,1,1,1,1
132,10025,2007-05-31,0.048872,3.503347e+08,3.340109e+08,00103110,2007-05,2007-04,11903.0,2007.0,...,0,0,0,0,1,1,1,1,1,1
133,10025,2007-06-29,0.008289,3.398255e+08,3.503347e+08,00103110,2007-06,2007-05,11903.0,2007.0,...,0,0,0,0,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
812602,93436,2022-12-30,-0.367334,3.897415e+11,6.148143e+11,88160R10,2022-12,2022-11,184996.0,2022.0,...,0,1,0,1,0,1,0,1,0,1
812603,93436,2023-01-31,0.406235,5.480859e+11,3.897415e+11,88160R10,2023-01,2022-12,184996.0,2022.0,...,0,1,0,1,0,1,0,1,0,1
812604,93436,2023-02-28,0.187565,6.508877e+11,5.480859e+11,88160R10,2023-02,2023-01,184996.0,2023.0,...,0,1,0,1,0,1,0,1,0,1
812605,93436,2023-03-31,0.008507,6.575059e+11,6.508877e+11,88160R10,2023-03,2023-02,184996.0,2023.0,...,0,1,0,1,0,1,0,1,0,1


In [348]:
def portfolio_membership_counts(df, date_col="YearMonth", id_col=None):
    """
    Count portfolio membership per month for columns like:
      <signal>_(top|bottom)_(decile|quintile|tertile)[_industry]
      industry_<signal>_(top|bottom)_(decile|quintile|tertile)

    If id_col is None  -> count rows (firm-month obs).
    If id_col provided -> count unique ids (e.g., cusip, permno).
    """
    x = df.copy()
    x["_m_"] = _to_month_period(x[date_col])

    # detect top/bottom indicators (both suffix- and prefix-style), incl. tertile
    pat_suf = re.compile(r"^(?P<signal>.+)_(?P<leg>top|bottom)_(?P<bucket>decile|quintile|tertile)(?P<scope>_industry)?$")
    pat_pre = re.compile(r"^industry_(?P<signal>.+)_(?P<leg>top|bottom)_(?P<bucket>decile|quintile|tertile)$")

    cols = []
    for c in x.columns:
        m1, m2 = pat_suf.match(c), pat_pre.match(c)
        if not (m1 or m2):
            continue
        d = (m1 or m2).groupdict()
        scope = "industry" if (m1 and d.get("scope")) or m2 else "overall"
        key = f"{d['signal']}__{scope}__{d['bucket']}__{d['leg']}"
        cols.append((c, key))

    if not cols:
        return pd.DataFrame()

    # ensure numeric 0/1
    for c, _ in cols:
        x[c] = pd.to_numeric(x[c], errors="coerce").fillna(0).astype(int)

    rows = []
    for c, key in cols:
        mask = x[c] == 1
        if id_col:
            # unique ID count per month
            cnt = (x.loc[mask, ["_m_", id_col]]
                     .dropna(subset=[id_col])
                     .groupby("_m_")[id_col].nunique())
        else:
            # row count per month
            cnt = x.loc[mask].groupby("_m_").size()
        rows.append(cnt.rename(key))

    out = (pd.concat(rows, axis=1)
             .fillna(0)
             .astype(int)
             .reset_index()
             .rename(columns={"_m_": "YearMonth"}))
    return out
membership_rows = portfolio_membership_counts(df_with_flags, date_col="YearMonth")
membership_rows

,YearMonth,resid_n_car_1_material__overall__decile__bottom,resid_n_car_1_material__overall__decile__top,resid_n_car_1_material__overall__quintile__bottom,resid_n_car_1_material__overall__quintile__top,resid_n_car_1_material__overall__tertile__bottom,resid_n_car_1_material__overall__tertile__top,resid_n_car_1_material_industry__industry__decile__bottom,resid_n_car_1_material_industry__industry__decile__top,resid_n_car_1_material_industry__industry__quintile__bottom,...,resid_n_material__overall__quintile__bottom,resid_n_material__overall__quintile__top,resid_n_material__overall__tertile__bottom,resid_n_material__overall__tertile__top,resid_n_material_industry__industry__decile__bottom,resid_n_material_industry__industry__decile__top,resid_n_material_industry__industry__quintile__bottom,resid_n_material_industry__industry__quintile__top,resid_n_material_industry__industry__tertile__bottom,resid_n_material_industry__industry__tertile__top
0,2007-01,296,296,592,592,987,987,2410,2410,2467,...,592,592,987,987,2577,2577,2616,2616,2666,2666
1,2007-02,296,296,592,592,986,986,2191,2191,2273,...,592,592,986,986,2151,2151,2237,2237,2347,2347
2,2007-03,295,295,589,589,981,981,2237,2237,2312,...,589,589,981,981,2043,2043,2140,2140,2263,2263
3,2007-04,286,286,571,571,951,951,2761,2761,2768,...,571,571,951,951,2130,2130,2209,2209,2310,2310
4,2007-05,284,284,567,567,944,944,2616,2616,2637,...,567,567,944,944,2321,2321,2375,2375,2445,2445
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,2023-08,262,262,523,523,871,871,1117,1117,1279,...,523,523,871,871,554,554,778,778,1065,1066
200,2023-09,260,260,519,519,864,864,758,758,960,...,519,519,864,864,624,624,839,839,1114,1116
201,2023-10,259,259,517,517,861,861,942,942,1120,...,517,517,861,861,673,673,880,880,1146,1149
202,2023-11,257,257,513,513,854,854,651,651,862,...,513,513,854,854,604,604,820,820,1091,1092


In [364]:
def ls_excess_from_indicators(
    df: pd.DataFrame,
    date_col: str = "YearMonth",
    ret_col: str  = "ret_adj",
    rf_col: str   = "RF",
    weight_col: str = "me_lag",
):
    """
    Long–short *excess* returns (EW & VW) from binary flags with these names:
      overall:          <signal>_(top|bottom)_(tertile|quintile|decile)
      within industry:  <signal>_(top|bottom)_(tertile|quintile|decile)_industry
    (Legacy prefix 'industry_<signal>_...' is also supported → within industry.)

    Outputs per spec:
      <base>_<bucket>_ls_ew, <base>_<bucket>_ls_vw
    where <base> is a cleaned signal; prefixed with 'industry_' only for within-industry.
    """
    x = df.copy()

    # Month key → Period[M]
    if isinstance(x[date_col].dtype, pd.PeriodDtype):
        try:
            x["_m_"] = x[date_col].dt.asfreq("M")
        except Exception:
            x["_m_"] = x[date_col].astype("period[M]")
    else:
        x["_m_"] = pd.to_datetime(x[date_col].astype(str), errors="coerce").dt.to_period("M")

    # Excess returns & weights
    x["_rex_"] = (
        pd.to_numeric(x.get(ret_col, np.nan), errors="coerce")
        - pd.to_numeric(x.get(rf_col, 0.0), errors="coerce").fillna(0.0)
    )
    x["_w_"] = pd.to_numeric(x.get(weight_col, np.nan), errors="coerce")

    # Recognize columns: overall (no suffix) or within-industry (suffix _industry or legacy prefix)
    pat_suffix = re.compile(
        r"^(?P<signal>.+)_(?P<leg>top|bottom)_(?P<bucket>tertile|quintile|decile)"
        r"(?P<scope>_industry)?$"   # only '' or '_industry'
    )
    pat_prefix = re.compile(
        r"^(?P<pfx>industry_)(?P<signal>.+)_(?P<leg>top|bottom)_(?P<bucket>tertile|quintile|decile)$"
    )

    def scope_from(groups):
        s = groups.get("scope") or ""
        return "industry" if s == "_industry" else "overall"

    # Collect complete top/bottom pairs by (signal, scope, bucket)
    pairs = {}
    for c in x.columns:
        m = pat_suffix.match(c)
        if m:
            d = m.groupdict()
            key = (d["signal"], scope_from(d), d["bucket"])
        else:
            m = pat_prefix.match(c)
            if not m:
                continue
            d = m.groupdict()
            key = (d["signal"], "industry", d["bucket"])
        pairs.setdefault(key, {"top": None, "bottom": None})
        pairs[key][d["leg"]] = c

    specs = [
        (sig, scp, bkt, cols["top"], cols["bottom"])
        for (sig, scp, bkt), cols in pairs.items()
        if cols["top"] is not None and cols["bottom"] is not None
    ]
    if not specs:
        raise ValueError("No complete top/bottom pairs found for overall/industry.")

    # Clean signal name for output
    def _clean_signal(sig: str) -> str:
        s = sig
        s = re.sub(r"^n_", "", s)
        s = s.replace("resid_", "")
        s = s.replace("_material_24m", "").replace("_material", "")
        s = s.replace("_24m", "")
        return s.strip("_")

    # Safe mask conversion
    def _safe_mask(col: pd.Series) -> pd.Series:
        if col.dtype == bool:
            return col.astype("boolean").fillna(False)
        return pd.to_numeric(col, errors="coerce").eq(1).astype("boolean").fillna(False)

    # EW/VW on *excess* returns
    def _ew(mask: pd.Series) -> pd.Series:
        m = _safe_mask(mask)
        if not m.any():
            return pd.Series(dtype=float)
        return x.loc[m].groupby(x.loc[m, "_m_"])["_rex_"].mean()

    def _vw(mask: pd.Series) -> pd.Series:
        m = _safe_mask(mask)
        tmp = x.loc[m, ["_m_", "_rex_", "_w_"]].copy()
        if tmp.empty:
            return pd.Series(dtype=float)
        tmp["_w_"] = tmp["_w_"].clip(lower=0)
        sumw = tmp.groupby("_m_")["_w_"].sum()
        sumw = sumw.where(sumw > 0)
        tmp = tmp.join(sumw.rename("_sumw_"), on="_m_")
        tmp["_contrib_"] = tmp["_rex_"] * (tmp["_w_"] / tmp["_sumw_"])
        return tmp.groupby("_m_")["_contrib_"].sum()

    # Build LS series
    frames = []
    for sig, scp, bkt, top_col, bot_col in sorted(specs):
        ew_top, ew_bot = _ew(x[top_col]), _ew(x[bot_col])
        vw_top, vw_bot = _vw(x[top_col]), _vw(x[bot_col])

        idx = ew_top.index.union(ew_bot.index)
        ew_ls = ew_top.reindex(idx) - ew_bot.reindex(idx)

        idx_vw = vw_top.index.union(vw_bot.index)
        vw_ls = vw_top.reindex(idx_vw) - vw_bot.reindex(idx_vw)
        vw_ls = vw_ls.reindex(idx)

        base = _clean_signal(sig)
        if scp == "industry":
            base = f"industry_{base}"

        frames.append(pd.DataFrame({
            "YearMonth": idx,
            f"{base}_{bkt}_ls_ew": ew_ls.values,
            f"{base}_{bkt}_ls_vw": vw_ls.values,
        }))

    out = frames[0]
    for f in frames[1:]:
        out = out.merge(f, on="YearMonth", how="outer")

    return out.sort_values("YearMonth").reset_index(drop=True)


In [365]:
# df_with_indicators: your dataframe that already contains the *_top_* and *_bottom_* flags
ls_ret = ls_excess_from_indicators(df_with_flags)
ls_ret

,YearMonth,n_car_1_decile_ls_ew,n_car_1_decile_ls_vw,n_car_1_quintile_ls_ew,n_car_1_quintile_ls_vw,n_car_1_tertile_ls_ew,n_car_1_tertile_ls_vw,industry_n_car_1_industry_decile_ls_ew,industry_n_car_1_industry_decile_ls_vw,industry_n_car_1_industry_quintile_ls_ew,...,n_severity_quintile_ls_ew,n_severity_quintile_ls_vw,n_severity_tertile_ls_ew,n_severity_tertile_ls_vw,industry_n_severity_industry_decile_ls_ew,industry_n_severity_industry_decile_ls_vw,industry_n_severity_industry_quintile_ls_ew,industry_n_severity_industry_quintile_ls_vw,industry_n_severity_industry_tertile_ls_ew,industry_n_severity_industry_tertile_ls_vw
0,2007-01,-0.011229,0.027097,-0.014843,0.027896,-0.011261,0.025248,0.000045,0.001588,0.000093,...,-0.014225,0.033501,-0.011227,0.029062,-0.000108,0.003308,-0.000601,0.003803,-0.000118,0.003996
1,2007-02,0.018276,-0.002157,0.021310,-0.001504,0.014657,-0.001207,0.001708,-0.000694,0.001742,...,0.018685,-0.012774,0.014115,-0.011875,0.001400,-0.002785,0.001153,-0.002279,0.003302,-0.001360
2,2007-03,-0.001836,-0.004628,0.010708,-0.004001,0.002143,-0.004381,0.001191,0.000273,0.002395,...,0.007515,-0.009082,0.002086,-0.009182,-0.000169,-0.002034,0.000700,-0.001682,0.002182,-0.001747
3,2007-04,0.025260,-0.001381,0.037336,0.000259,0.031961,0.001010,0.001146,0.000764,0.002220,...,0.038647,-0.001019,0.036022,-0.000053,0.000231,0.001047,0.002784,0.002097,0.006848,0.002910
4,2007-05,-0.038253,0.010758,-0.026296,0.011061,-0.028514,0.009879,-0.002885,0.012538,-0.004025,...,-0.030220,-0.006281,-0.030057,-0.006692,-0.004440,0.010359,-0.005455,0.008264,-0.008415,0.004777
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,2023-08,-0.016187,-0.017443,-0.012397,-0.019275,0.001567,-0.028272,-0.008856,-0.006299,-0.022515,...,-0.015686,-0.013675,-0.002751,-0.013220,-0.009867,-0.010369,-0.023194,-0.007661,-0.013236,-0.011414
200,2023-09,0.024368,-0.018283,0.017147,-0.023984,0.013342,-0.028685,0.002036,-0.018823,-0.002361,...,0.014403,-0.036036,0.008371,-0.035278,0.003485,-0.035523,-0.003190,-0.033119,0.000742,-0.030220
201,2023-10,0.012196,0.010117,-0.000756,0.016295,0.002297,0.021754,-0.025815,-0.004598,-0.007943,...,-0.000796,0.011132,0.002557,0.005583,-0.015791,-0.005917,-0.013256,-0.004923,0.004643,-0.010510
202,2023-11,0.022722,0.041872,-0.013532,0.046603,-0.027179,0.042992,0.051698,0.040939,0.033875,...,-0.001902,0.052779,-0.023843,0.050538,0.041676,0.040156,0.027237,0.038589,0.020778,0.036515


In [366]:
df_ls.to_csv('Output/portfolio_residual_rolling.csv', index=False)

# 4. Portfolio Returns

In [844]:
# ----------------------------
# 0) Helpers
# ----------------------------

def choose_primary_class(df, by=['cusip','date']):
    """
    If multiple PERMNOs share the same cusip8 at a date,
    keep the one with the largest lagged market cap (me_lag).
    """
    df = df.copy()
    df['_rank'] = df.groupby(by)['me_lag'].rank(method='first', ascending=False)
    out = df.loc[df['_rank'] == 1].drop(columns=['_rank'])
    return out

In [845]:
# ----------------------------
# 1) Prepare CRSP monthly panel
# ----------------------------
crsp = crsp_with_cusip.copy()
crsp['date'] = pd.to_datetime(crsp['date'])


# If same cusip8 maps to multiple permnos per month, keep largest me_lag
crsp_1class = choose_primary_class(crsp, by=['cusip','date'])

In [846]:
# ----------------------------
# 2) Prepare factors
# ----------------------------
ff = ff.copy()
ff['date'] = pd.to_datetime(ff['date']) + MonthEnd(0)

In [847]:
def compute_all_quintile_portfolio_returns(panel):
    """
    Computes EW/VW time series for each flag column and long–short spreads per 'portfolio type':
      - Quintile: topQ_* minus botQ_*
      - Decile:   topD_* minus botD_*
      - Incident: noInc_* minus inc_*

    Expects columns:
      - 'date' (datetime-like), 'ret_adj' (returns), optional 'me_lag' (for VW)
      - Portfolio flags: topQ_/botQ_, topD_/botD_, noInc_/inc_
    """
    panel = panel.copy()
    panel['date'] = pd.to_datetime(panel['date'])

    # 1) detect flags for all three portfolio types
    top_flags = [c for c in panel.columns if c.startswith(('topQ_', 'topD_', 'noInc_'))]
    bot_flags = [c for c in panel.columns if c.startswith(('botQ_', 'botD_', 'inc_'))]

    if not top_flags and not bot_flags:
        raise ValueError("No portfolio flag columns found (topQ_/botQ_/topD_/botD_/noInc_/inc_).")

    # 2) helper: return series for one flag (EW or VW)
    def _flag_returns(df, flag_col, weight='EW'):
        p = df.loc[df[flag_col] == 1].copy()
        name = f"{flag_col}_{weight}"
        if p.empty:
            return pd.Series(dtype=float, name=name)
        if weight == 'EW':
            return p.groupby('date')['ret_adj'].mean().rename(name)
        # VW uses lagged market equity
        p = p.dropna(subset=['me_lag']).copy()
        if p.empty:
            return pd.Series(dtype=float, name=name)
        wsum = p.groupby('date')['me_lag'].sum().rename('W')
        p = p.merge(wsum, on='date', how='left')
        p['w'] = np.where(p['W'] > 0, p['me_lag'] / p['W'], np.nan)
        return (p['w'] * p['ret_adj']).groupby(p['date']).sum().rename(name)

    # 3) compute EW/VW for every flag
    parts = []
    for flag in sorted(set(top_flags + bot_flags)):
        parts.append(_flag_returns(panel, flag, 'EW'))
        parts.append(_flag_returns(panel, flag, 'VW'))
    ts_flags = pd.concat(parts, axis=1).sort_index()

    # 4) parse (type, base) from a flag name so "col split" works for all types
    # Returns (ptype, side, base) where:
    #   ptype in {'Q','D','INC'}; side in {'top','bot'}
    def _parse(col):
        if col.startswith('topQ_'):  return ('Q',   'top', col[5:])
        if col.startswith('botQ_'):  return ('Q',   'bot', col[5:])
        if col.startswith('topD_'):  return ('D',   'top', col[5:])
        if col.startswith('botD_'):  return ('D',   'bot', col[5:])
        if col.startswith('noInc_'): return ('INC', 'top', col[6:])  # long leg = no incident
        if col.startswith('inc_'):   return ('INC', 'bot', col[4:])  # short leg = incident
        return (None, None, None)

    # build maps: {(ptype, base): column_name}
    top_map, bot_map = {}, {}
    for c in top_flags:
        ptype, side, base = _parse(c)
        if ptype and side == 'top':
            top_map[(ptype, base)] = c
    for c in bot_flags:
        ptype, side, base = _parse(c)
        if ptype and side == 'bot':
            bot_map[(ptype, base)] = c

    # 5) long–short per (ptype, base)
    common_keys = sorted(set(top_map).intersection(bot_map))
    for ptype, base in common_keys:
        tflag, bflag = top_map[(ptype, base)], bot_map[(ptype, base)]
        # EW spread
        t_ew = f"{tflag}_EW"; b_ew = f"{bflag}_EW"
        if t_ew in ts_flags.columns and b_ew in ts_flags.columns:
            ts_flags[f"LS_{ptype}_EW_{base}"] = ts_flags[t_ew] - ts_flags[b_ew]
        # VW spread
        t_vw = f"{tflag}_VW"; b_vw = f"{bflag}_VW"
        if t_vw in ts_flags.columns and b_vw in ts_flags.columns:
            ts_flags[f"LS_{ptype}_VW_{base}"] = ts_flags[t_vw] - ts_flags[b_vw]

    return ts_flags

In [856]:
ts_all = compute_all_quintile_portfolio_returns(df_ports_q)
ts_all

,botQ_resid_n_car_1_material_EW,botQ_resid_n_car_1_material_VW,botQ_resid_n_car_5_material_EW,botQ_resid_n_car_5_material_VW,botQ_resid_n_material_EW,botQ_resid_n_material_VW,botQ_resid_n_reach_material_EW,botQ_resid_n_reach_material_VW,botQ_resid_n_severity_material_EW,botQ_resid_n_severity_material_VW,...,LS_Q_EW_resid_n_car_1_material,LS_Q_VW_resid_n_car_1_material,LS_Q_EW_resid_n_car_5_material,LS_Q_VW_resid_n_car_5_material,LS_Q_EW_resid_n_material,LS_Q_VW_resid_n_material,LS_Q_EW_resid_n_reach_material,LS_Q_VW_resid_n_reach_material,LS_Q_EW_resid_n_severity_material,LS_Q_VW_resid_n_severity_material
date,,,,,,,,,,,,,,,,,,,,,
2009-01-30,-0.043333,-0.082482,-0.053108,-0.130878,-0.082064,-0.103613,-0.085532,-0.10103,-0.069587,-0.081988,...,-0.017887,0.003828,-0.000732,0.066437,0.015229,0.040192,0.024623,0.03275,0.005983,0.003026
2009-02-27,-0.089405,-0.065724,-0.085311,-0.119433,-0.092809,-0.049271,-0.092382,-0.0575,-0.083065,-0.05655,...,-0.007929,-0.011895,-0.007843,0.050074,0.001006,-0.03439,-0.002811,-0.017906,-0.001342,-0.018585
2009-03-31,0.140397,0.084326,0.167861,0.066808,0.154676,0.076959,0.155781,0.087619,0.151328,0.08739,...,0.011009,0.00786,-0.016864,0.02945,-0.001941,0.032217,0.00428,0.006709,0.001931,0.006093
2009-04-30,0.298713,0.061675,0.316874,0.063588,0.31296,0.049921,0.309689,0.094739,0.312832,0.049294,...,-0.001111,0.009046,-0.030423,0.005339,-0.018159,0.03076,-0.027615,-0.030923,-0.025537,0.02694
2009-05-29,0.075072,0.02042,0.051701,0.03223,0.064956,0.023868,0.054201,0.010679,0.050629,0.013049,...,-0.045585,0.0067,-0.021639,-0.006828,-0.038167,-0.000082,-0.019288,0.018321,-0.0189,0.015722
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-31,-0.06273,0.00596,-0.069024,0.001336,-0.065196,0.000988,-0.070525,-0.000579,-0.067456,-0.000881,...,0.010752,-0.023016,0.014314,-0.016058,0.012974,-0.019968,0.016956,-0.025519,0.014139,-0.022284
2023-09-29,-0.072279,-0.048823,-0.079409,-0.04674,-0.066178,-0.040277,-0.067034,-0.041843,-0.065827,-0.039111,...,0.004012,-0.000909,0.012576,-0.002129,-0.006099,-0.022383,-0.006309,-0.015935,-0.006658,-0.018048
2023-10-31,-0.075596,-0.005215,-0.071122,0.003208,-0.080543,0.021351,-0.07693,0.018555,-0.075831,0.02084,...,0.015726,0.006191,0.016606,-0.006338,0.023204,-0.055873,0.025533,-0.046826,0.0203,-0.056175


In [857]:
def add_factors_and_excess(ts_all, ff, rf_col='RF', subtract_rf_ls=False):
    """
    Merge ALL factors from ff into ts_all and create *_ex series.
    Returns (ts_with_factors, factor_cols_detected)
    """
    # 1) merge all factor columns by date
    fac_cols_all = [c for c in ff.columns if c != 'date']
    ts = ts_all.merge(ff[['date'] + fac_cols_all], on='date', how='left').copy()

    # 2) identify legs & LS
    leg_cols = [c for c in ts.columns
                if (c.endswith('_EW') or c.endswith('_VW')) and not c.startswith('LS_')]
    ls_cols  = [c for c in ts.columns if c.startswith('LS_')]

    # 3) build excess for legs
    for col in leg_cols:
        ts[f"{col}_ex"] = ts[col] - ts[rf_col]

    # 4) LS excess (optional)
    if subtract_rf_ls:
        for col in ls_cols:
            ts[f"{col}_ex"] = ts[col] - ts[rf_col]

    # 5) detect factor columns actually present
    wanted = ['MKT_RF','SMB','HML','RMW','CMA','MOM']
    factor_cols = [c for c in wanted if c in ts.columns]
    return ts, factor_cols

# usage
ts_with_ex, factor_cols = add_factors_and_excess(ts_all, ff, rf_col='RF', subtract_rf_ls=False)


In [858]:
ts_with_ex

,date,botQ_resid_n_car_1_material_EW,botQ_resid_n_car_1_material_VW,botQ_resid_n_car_5_material_EW,botQ_resid_n_car_5_material_VW,botQ_resid_n_material_EW,botQ_resid_n_material_VW,botQ_resid_n_reach_material_EW,botQ_resid_n_reach_material_VW,botQ_resid_n_severity_material_EW,...,topQ_resid_n_car_1_material_EW_ex,topQ_resid_n_car_1_material_VW_ex,topQ_resid_n_car_5_material_EW_ex,topQ_resid_n_car_5_material_VW_ex,topQ_resid_n_material_EW_ex,topQ_resid_n_material_VW_ex,topQ_resid_n_reach_material_EW_ex,topQ_resid_n_reach_material_VW_ex,topQ_resid_n_severity_material_EW_ex,topQ_resid_n_severity_material_VW_ex
0,2009-01-30,-0.043333,-0.082482,-0.053108,-0.130878,-0.082064,-0.103613,-0.085532,-0.10103,-0.069587,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,2009-02-27,-0.089405,-0.065724,-0.085311,-0.119433,-0.092809,-0.049271,-0.092382,-0.0575,-0.083065,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,2009-03-31,0.140397,0.084326,0.167861,0.066808,0.154676,0.076959,0.155781,0.087619,0.151328,...,0.151206,0.091987,0.150797,0.096058,0.152535,0.108976,0.159861,0.094128,0.153058,0.093283
3,2009-04-30,0.298713,0.061675,0.316874,0.063588,0.31296,0.049921,0.309689,0.094739,0.312832,...,0.297502,0.070621,0.286351,0.068827,0.294701,0.080581,0.281973,0.063717,0.287195,0.076134
4,2009-05-29,0.075072,0.02042,0.051701,0.03223,0.064956,0.023868,0.054201,0.010679,0.050629,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,2023-08-31,-0.06273,0.00596,-0.069024,0.001336,-0.065196,0.000988,-0.070525,-0.000579,-0.067456,...,-0.056477,-0.021556,-0.05921,-0.019222,-0.056722,-0.02348,-0.05807,-0.030598,-0.057816,-0.027664
176,2023-09-29,-0.072279,-0.048823,-0.079409,-0.04674,-0.066178,-0.040277,-0.067034,-0.041843,-0.065827,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
177,2023-10-31,-0.075596,-0.005215,-0.071122,0.003208,-0.080543,0.021351,-0.07693,0.018555,-0.075831,...,-0.06457,-0.003724,-0.059216,-0.00783,-0.062039,-0.039222,-0.056097,-0.032971,-0.060232,-0.040035
178,2023-11-30,0.096233,0.056254,0.095974,0.05363,0.106476,0.088386,0.090968,0.067366,0.087679,...,0.12399,0.111909,0.120508,0.102186,0.126973,0.115612,0.128897,0.108572,0.125555,0.109295


In [859]:
ts_with_ex.to_csv('Output/portfolio_returns_quintiles.csv', index=False)

# 5. Sample Selection

In [ ]:
# Reprisk firms


In [599]:
firms = pd.read_csv('data/panel_firms_quintile.csv')

In [600]:
firms.notna().sum()

Unnamed: 0                        84493
permno                            84493
date                              84493
yearprior                         84493
ret_adj                           84493
me                                84493
me_lag                            84486
cusip                             84493
year                              84493
Codified SICS Sector              84493
SICS Codified Industry            84493
size                              84493
btm                               84493
leverage                          84493
rd_intensity                      84493
ad_intensity                      84493
roa                               84493
InstOwn_Perc                      84493
n_material                        84493
n_car_1_material                  84493
n_car_5_material                  84493
n_reach_material                  84493
n_rri_material                    84493
n_severity_material               84493
dateff                            84493


In [601]:
firms['cusip'].nunique()

904

In [478]:
# aggregate to yearprior level
firms.groupby('yearprior')['cusip'].nunique().sum()

np.int64(7165)

In [479]:
firms.groupby('yearprior')['cusip'].nunique()

yearprior
2008    387
2009    397
2010    409
2011    411
2012    416
2013    428
2014    442
2015    452
2016    453
2017    449
2018    475
2019    479
2020    469
2021    478
2022    516
2023    504
Name: cusip, dtype: int64